In [1]:
# includes
import pandas as pd
import numpy as np
import math
from random import sample
from matplotlib import pyplot as plt


In [2]:
# read in the file
df = pd.read_csv('mutations_3.csv')

In [1710]:
# rename unnamed column
df.rename(columns={'Unnamed: 0':'Patients'}, inplace=True)
df

,Patients,FARP1_GRCh37_13:99092237-99092237_Frame-Shift-Del_DEL_G-G--,QKI_GRCh37_6:163987695-163987695_Intron_DEL_T-T--,NHLRC2_GRCh37_10:115662308-115662308_Frame-Shift-Del_DEL_A-A--,PDZD8_GRCh37_10:119042898-119042898_Frame-Shift-Del_DEL_T-T--,EBF3_GRCh37_10:131666125-131666125_Frame-Shift-Del_DEL_C-C--,MASTL_GRCh37_10:27459272-27459272_Frame-Shift-Del_DEL_A-A--,PTCHD3_GRCh37_10:27687763-27687763_Frame-Shift-Del_DEL_A-A--,PFKP_GRCh37_10:3151612-3151612_Silent_SNP_C-C-T,CREM_GRCh37_10:35456559-35456559_Intron_DEL_T-T--,...,PHF2_GRCh37_9:96422642-96422642_Missense-Mutation_SNP_C-C-A,USP51_GRCh37_X:55513209-55513209_3'UTR_DEL_T-T--,CDH23_GRCh37_10:73405660-73405660_Missense-Mutation_SNP_G-G-A,SLC26A11_GRCh37_17:78195370-78195370_Missense-Mutation_SNP_C-C-T,MME_GRCh37_3:154866382-154866382_Missense-Mutation_SNP_A-A-T_A-A-C,CADPS_GRCh37_3:62459960-62459960_Missense-Mutation_SNP_G-G-T,SCN4A_GRCh37_17:62049998-62049998_Silent_SNP_G-G-A,FPR2_GRCh37_19:52272624-52272624_Missense-Mutation_SNP_G-G-A,OLFM3_GRCh37_1:102302454-102302454_Missense-Mutation_SNP_A-A-C_A-A-G,sum_row
0,C1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,6.0
1,C2,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,178.0
2,NC1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,1.0
3,C3,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,3.0
4,NC2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,TN (True Negative),106.000000,105.000000,106.000000,106.000000,106.000000,105.0,106.000000,106.000000,106.000000,...,106.000000,105.0,105.0,105.0,105.0,104.000000,105.0,104.000000,106.000000,NaN
173,FP (False Positive),0.000000,1.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,...,0.000000,1.0,1.0,1.0,1.0,2.000000,1.0,2.000000,0.000000,NaN
174,FN (False Negative),54.000000,53.000000,54.000000,54.000000,52.000000,55.0,53.000000,54.000000,54.000000,...,54.000000,55.0,55.0,55.0,55.0,56.000000,55.0,56.000000,54.000000,NaN
175,TP-FP,2.000000,2.000000,2.000000,2.000000,4.000000,0.0,3.000000,2.000000,2.000000,...,2.000000,0.0,0.0,0.0,0.0,-2.000000,0.0,-2.000000,2.000000,NaN


In [81]:
# Homework 4 - Part 1

In [82]:
# question 1 - show the top 10 values of TP - FP
# make a new dataframe to only have row 175(TP-FP) and all the features
q1res = df.iloc[175, 1:1411]

# Sort the new dataframe by decending order and show the top 10
q1res.sort_values(ascending=False).head(10)

DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--        12.0
ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--     10.0
PGM5_GRCh37_9:70993145-70993145_Missense-Mutation_SNP_A-A-G       10.0
ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--     10.0
RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--       10.0
UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--        9.0
LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del_DEL_T-T--           8.0
UPF3A_GRCh37_13:115057211-115057211_Frame-Shift-Del_DEL_A-A--      7.0
TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_C-C-T                7.0
KIAA0195_GRCh37_17:73491063-73491063_Frame-Shift-Del_DEL_C-C--     7.0
Name: 175, dtype: object

In [83]:
# question 2 - this was answered in the previous table
# DOCK3 with the value of 12

In [84]:
# question 3 - split the patients up by if they have DOCK3 or not
# If yes = group A, no = group B

# makeing the temp dataframe
tmpdf = pd.DataFrame(columns=['GroupA', 'GroupB'])
count = 0 # iter
tmp = 0 # for groupA pos
tmp2 = 0 # for groupb pos
for i in df.iloc[0:162]['DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--']:
    if i == 1:
        x = df.iloc[count, 0]
        tmpdf.at[tmp, 'GroupA'] = x
        tmp += 1
    elif i == 0:
        x = df.iloc[count, 0]
        tmpdf.at[tmp2, 'GroupB'] = x
        tmp2 += 1
    count = count + 1

# print out the new dataframe
tmpdf.head(15)

,GroupA,GroupB
0,C2,C1
1,C12,NC1
2,C24,C3
3,C32,NC2
4,C36,NC3
5,C39,NC4
6,C44,NC5
7,C48,NC6
8,C50,NC7
9,C51,NC8


In [85]:
# question 3 - 
# make 4 columns in the dataframe tmpdf for TP TN FP FN

tmpdf2 = pd.DataFrame(columns=['Group1', 'Group2','GroupA', 'GroupB'])
tmpdf2['Group1'] = df[0:162]['Patients']
tmpdf2['GroupA'] = tmpdf['GroupA']
tmpdf2['GroupB'] = tmpdf['GroupB']
count = 0
for i in tmpdf2['Group1']:
    if i.find('N') == 0: # then we found a NC so we make a 0
        tmpdf2.at[count, 'Group2'] = 0
    else:
        tmpdf2.at[count, 'Group2'] = 1
    count += 1

fp_total, tp_total, fn_total, tn_total = 0,0,0,0
# go through people with the DOCK3 mutation
for i in tmpdf2[0:12]['GroupA']:
    if i.find('N') == 1: # found a NC that has cancer so add it to FP
        fp_total += 1
    elif i.find('C') == 1: # the people with cancer were correctly diagnosed so add to tp
        tp_total += 1

# go through the people without DOCK3 mutation
for i in tmpdf2[0:150]['GroupB']:
    if i.find('N') == 0: # we found a correctly labeled NC so we add it to TN
        tn_total += 1
    elif i.find('C') == 0: # found a wrongly labeled cancer patient so we add to fn
        fn_total += 1
        
# debug
#print(fp_total, ' ', tp_total, ' ', tn_total, ' ', fn_total)
 
# load up the confusion matrix

data = [{'FP' : fp_total, 'TP' : tp_total, 'TN' : tn_total, 'FN' : fn_total }]
confusionM_Dock3 = pd.DataFrame(data)

# output
confusionM_Dock3


,FP,TP,TN,FN
0,0,0,106,44


In [86]:
## Part 2
# Q1: show FP - TP for GroupA, show top 10 features. Select the top feature and make a confusion matrix with that
# rename GroupA to Patients for the merge
tmpdf.rename(columns={'GroupA':'Patients'}, inplace=True)

# merge on GroupA with the df to only get groupA rows
groupadf = pd.merge(tmpdf['Patients'], df, on = ['Patients', 'Patients'])

# delete columns we dont need
del groupadf['sum_row']
del groupadf['DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--']

# now get TP FP TN FN
groupadf.loc[12, 'Patients'] = 'TP' # the amount of 1's for C patients
groupadf.loc[13, 'Patients'] = 'FP' # the amount of 1's for NC patients
groupadf.loc[14, 'Patients'] = 'TP-FP' # TP - FP

tp, fp = 0, 0
for i in range(1,1410):
    for j in range(0,12):
        # get the patient
        cur_pat = groupadf.iloc[j, 0]
        # if there is a noncancer patient
        if(cur_pat.find('N') == 0):
            if(groupadf.iloc[j, i] == 1):
                fp += 1
        else:
            if(groupadf.iloc[j, i] == 1):
                tp += 1
        groupadf.iloc[12, i] = tp
        groupadf.iloc[13, i] = fp
        groupadf.iloc[14, i] = tp - fp
    
    # reset var
    tp = 0
    fp = 0

# Sorted by TP-FP in a new dataframe
sgroupadf = groupadf.iloc[14, 1:1410].sort_values(ascending=False)

# output the top 10 features of TP - FP
sgroupadf.head(10)

UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--      7.0
RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--      7.0
ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--    6.0
PGM5_GRCh37_9:70993145-70993145_Missense-Mutation_SNP_A-A-G      6.0
LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del_DEL_T-T--         6.0
TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_C-C-T              5.0
CCAR2_GRCh37_8:22472975-22472975_Frame-Shift-Del_DEL_C-C--       5.0
ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--    5.0
RNF128_GRCh37_X:105937256-105937256_Frame-Shift-Del_DEL_T-T--    4.0
PHF2_GRCh37_9:96422612-96422612_Frame-Shift-Del_DEL_A-A--        4.0
Name: 14, dtype: object

In [87]:
# make a confusion matrix with the feature:
# UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--
groupa1 = pd.DataFrame(columns=['Patients', 'UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--', 'GroupA1', 'GroupA2'])

# get the patients and get the feature
groupa1['Patients'] = groupadf.loc[0:11,'Patients']
groupa1['UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--'] = groupadf.loc[0:11, 'UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--']

# seperate the 2 groups into GroupA1 and GroupA2
count = 0 # iter
tmp = 0 # for groupA1 pos
tmp2 = 0 # for groupA2 pos
for i in groupa1['UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--']:
    if i == 1:
        x = groupa1.iloc[count, 0]
        groupa1.at[tmp, 'GroupA1'] = x
        tmp += 1
    elif i == 0:
        x = groupa1.iloc[count, 0]
        groupa1.at[tmp2, 'GroupA2'] = x
        tmp2 += 1
    count = count + 1


# now do the confusion matrx
# if C == 1 then tp += 1 if c == 0 then fp == 1
tp, fp = 0,0
for i in groupa1['UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--']:
    if(i == 1):
        tp += 1
    else:
        fp += 1

# should sum to 12
print("The confusion matrix for the feature UBR5 is:\nTP: ", tp, " FN ", 0, " TN: ", 0, " FP: ", fp)
groupa1

The confusion matrix for the feature UBR5 is:
TP:  7  FN  0  TN:  0  FP:  5


,Patients,UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--,GroupA1,GroupA2
0,C2,1.0,C2,C12
1,C12,0.0,C24,C32
2,C24,1.0,C36,C50
3,C32,0.0,C39,C51
4,C36,1.0,C44,C54
5,C39,1.0,C48,NaN
6,C44,1.0,C55,NaN
7,C48,1.0,NaN,NaN
8,C50,0.0,NaN,NaN
9,C51,0.0,NaN,NaN


In [88]:
# Q2 - do Q1 but for groupb
# confusion matrix should sum to 150

# rename GroupB to Patients for the merge
dltr = tmpdf2.iloc[:,[2, 3]]
dltr.rename(columns={'GroupB':'Patients'}, inplace=True)

groupbdf = pd.merge(dltr, df, on=['Patients','Patients'])

# delete unused stuff
del groupbdf['GroupA']
del groupbdf['sum_row']
del groupbdf['DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--']
del dltr

# make rows for TP FP
groupbdf.loc[150, 'Patients'] = 'TP' # the amount of 1's for C patients
groupbdf.loc[151, 'Patients'] = 'FP' # the amount of 1's for NC patients
groupbdf.loc[152, 'Patients'] = 'TP-FP' # TP - FP


tp, fp = 0, 0
for i in range(1,1410):
    for j in range(0,150):
        cur_pat = groupbdf.iloc[j, 0]
        # if its NC
        if(cur_pat.find('N') == 0):
            if(groupbdf.iloc[j,i] == 1):
                fp += 1
        else: # it is a cancer patient and they have cancer
            if(groupbdf.iloc[j, i] == 1):
                tp += 1
        # insert vars for that column
        groupbdf.iloc[150, i] = tp
        groupbdf.iloc[151, i] = fp
        groupbdf.iloc[152, i] = tp - fp
    # reset vars
    tp = 0
    fp = 0

# print the dataframe
groupbdf.iloc[152, 1:1410].sort_values(ascending=False).head(10)



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--    5.0
ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--    4.0
PGM5_GRCh37_9:70993145-70993145_Missense-Mutation_SNP_A-A-G      4.0
UPF3A_GRCh37_13:115057211-115057211_Frame-Shift-Del_DEL_A-A--    4.0
ZC3H18_GRCh37_16:88691141-88691141_Frame-Shift-Del_DEL_C-C--     4.0
BMPR2_GRCh37_2:203420130-203420130_Frame-Shift-Del_DEL_A-A--     4.0
IWS1_GRCh37_2:128238676-128238676_Frame-Shift-Del_DEL_T-T--      4.0
ZNF330_GRCh37_4:142143532-142143532_Frame-Shift-Del_DEL_A-A--    4.0
ARL4D_GRCh37_17:41477713-41477713_3'UTR_DEL_C-C--                4.0
KDM1B_GRCh37_6:18222307-18222307_3'UTR_DEL_A-A--                 4.0
Name: 152, dtype: object

In [89]:
# make a confusion matrix with the top feature:
# ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--
groupb1 = pd.DataFrame(columns=['Patients', 'ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--', 'GroupB1', 'GroupB2'])

# getting the patients and feature
groupb1['Patients'] = groupbdf.loc[0:149, 'Patients']
groupb1['ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--'] = groupbdf.loc[0:149, 'ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--']

# seperate the group by if they have the feature
# also making the confusion matrix
idx1, idx2, counter = 0, 0, 0
for i in groupb1['ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--']:
    if i == 1:
        x = groupb1.iloc[counter, 0]
        groupb1.at[idx1, 'GroupB1'] = x
        idx1 += 1
    else:
        x = groupb1.iloc[counter, 0]
        groupb1.at[idx2, 'GroupB2'] = x
        idx2 += 1
    counter += 1
        
fn, tn, fp, tp = 0, 0, 0, 0
# this group all has the mutation
for i in groupb1.loc[0:6,'GroupB1']:
    if(i.find('NC') == 0):
        fp += 1
    elif(i.find('C') == 0): # its a cancer patient
        tp += 1
# this group all doesnt have it
for i in groupb1.loc[0:142, 'GroupB2']:
    if(i.find('NC') == 0):
        tn += 1
    elif(i.find('C') == 0):
        fn += 1

# print out
print('The confusion matrix for ACVR2 is:\nTP: ', tp, " FN ", fn, " TN: ", tn, " FP: ", fp)
groupb1

The confusion matrix for ACVR2 is:
TP:  6  FN  38  TN:  105  FP:  1


,Patients,ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--,GroupB1,GroupB2
0,C1,0.0,C5,C1
1,NC1,0.0,C13,NC1
2,C3,0.0,C15,C3
3,NC2,0.0,C30,NC2
4,NC3,0.0,C31,NC3
...,...,...,...,...
145,NC103,0.0,NaN,NaN
146,NC104,0.0,NaN,NaN
147,NC105,0.0,NaN,NaN
148,NC106,0.0,NaN,NaN


In [90]:
# question 10 - Given a mutation check if its in the decision tree then return C or NC
# given a patient
patient_input = input("Give the name of the Patient: ")
input_row = df.loc[df['Patients'] == patient_input]
# see if it has root
while(patient_input != 'quit'):
    if(input_row['DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--'].values == 1):
        # it has root check if it has UBR5
        if(input_row['UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--'].values == 1):
            print('Your inputted patient (', patient_input, ') is predicted to be: C\n')
        else:
            print('Your inputted patient (', patient_input, ') is predicted to be: NC\n')
    else:
        if(input_row['ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--'].values == 1):
            print('Your inputted patient (', patient_input, ') is predicted to be: C\n')
        else:
            print('Your inputted patient (', patient_input, ') is predicted to be: NC\n')
    del patient_input
    del input_row
    patient_input = input("Give the name of the Patient: ")
    input_row = df.loc[df['Patients'] == patient_input]


In [91]:
############################################# HOMEWORK 5 ##################################################################
# question 1 - compute the accuracy for the mutations then show the top 10 of the highest
hw5df = df

# add a row for accuracy
hw5df.loc[177, 'Patients'] = ['Accuracy']

# calculate accuracy by: (TP + TN) / 162

for i in range(1,1410):
    hw5df.iloc[177, i] = (hw5df.iloc[171, i] + hw5df.iloc[172, i]) / 162 # 162 per P + N (pos + n patients)

In [92]:
# top 10 accuracy ratings
hw5df.iloc[177, 1:1410].sort_values(ascending=False).head(10)

DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--        0.728395
ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--     0.716049
PGM5_GRCh37_9:70993145-70993145_Missense-Mutation_SNP_A-A-G       0.716049
ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--     0.716049
RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--       0.716049
UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--       0.709877
LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del_DEL_T-T--          0.703704
UPF3A_GRCh37_13:115057211-115057211_Frame-Shift-Del_DEL_A-A--     0.697531
TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_C-C-T               0.697531
KIAA0195_GRCh37_17:73491063-73491063_Frame-Shift-Del_DEL_C-C--    0.697531
Name: 177, dtype: object

In [93]:
# question 2 - go through all mutations 

hw5class = df[0:162]
del hw5class['sum_row']

# pass it the dataframe and the index of the patient (index is x variable)
def classifer(hw5groupa, index):
        if(hw5class.loc[index, 'DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--'] == 1):
            if(hw5class.loc[index, 'UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--'] == 1):
                return 'C'
            else:
                return 'NC'
        else:
            if(hw5class.loc[index, 'ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--'] == 1):
                return 'C'
            else:
                return 'NC'

# goes through all patients and classifies them, puts into a column
for i in range(0,162):
        hw5class.loc[i, 'classifier'] = classifer(hw5class, i)
        
# make a confusion matrix of comparing the classifer vs actual 
tp, tn = 0, 0
for i in range(0, 162):
    x = hw5class.iat[i, 0]
    y = hw5class.iat[i, 1411]
    if x.find('C') == 0 and y.find('C') == 0:
        tp += 1
    if x.find('NC') == 0 and y.find('NC') == 0:
        tn += 1
        
fp = 106 - tn
fn = 56 - tp
# (tp + tn) / p + n
accuracy = ((tp + tn) / 162)
# tp / (tp + fn)
sensitivity = tp / (tp + fn)
# tn / (fp + tn)
specificity = tn / (fp + tn)
# tp / (tp + fp)
precision = tp / (tp + fp)
# fn / (tp + fn)
missrate = fn / (tp + fn)
# fp / (tp + fp)
fdr = fp / (tp + fp)
# fn / (fn + tn)
falseomrate = fn / (fn + tn)


print("Classifier Stats:\nTP: ", tp, ' fp: ', fp, ' tn: ', tn, ' fn: ', fn, ' Accuracy: ', accuracy, ' Sensitivity: ', sensitivity)
print('Specificity', specificity, ' Precision: ', precision, ' Miss Rate: ', missrate, '\nFalse Discovery Rate: ', fdr, ' False Ommision Rate: ', falseomrate)

Classifier Stats:
TP:  13  fp:  1  tn:  105  fn:  43  Accuracy:  0.7283950617283951  Sensitivity:  0.23214285714285715
Specificity 0.9905660377358491  Precision:  0.9285714285714286  Miss Rate:  0.7678571428571429 
False Discovery Rate:  0.07142857142857142  False Ommision Rate:  0.2905405405405405


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [94]:
############################################ HW 6 ##########################################################################
# get the samples randomly mixed up
tmpdf3 = df.iloc[0:162, 0:1411].sample(n=162, replace=False)

# reset the indexes
tmpdf3.reset_index(drop=True)

# section it into 3 groups
onethird = tmpdf3[0:54]
twothird = tmpdf3[54:108]
threethird = tmpdf3[108:162]

# reset the indexes on them
onethird = onethird.reset_index(drop=True)
twothird = twothird.reset_index(drop=True)
threethird = threethird.reset_index(drop=True)

In [95]:
# make a new row at the bottom for tp and fp values
def setconfusion(confusiondf):
    # makes 2 new rows for TP and FP
    confusiondf.loc[confusiondf.shape[0], 'Patients'] = 'TP'
    confusiondf.loc[confusiondf.shape[0], 'Patients'] = 'FP'
    
    # loop through columns
    patientcounter = 0;
    for i in range(1, confusiondf.shape[1]):  # go through 1410
        tp, fp, Ccount, NCcount = 0, 0, 0, 0
        for j in range(0, confusiondf.shape[0]): # go through 108
            # if that row is a cancer patient then TP++
            # if that row is a NC then FP++
            name = confusiondf.iloc[j, 0]
            # print(name)
            if name.find('NC') == 0: 
                NCcount += 1
                # we found a NC
                if confusiondf.iloc[j, i] == 1:
                    fp += 1
            elif name.find('C') == 0:
                Ccount += 1
                if confusiondf.iloc[j, i] == 1:
                    tp += 1
            if j + 1 == confusiondf.shape[0]:
                confusiondf.iloc[confusiondf.shape[0] - 2, i] = tp
                confusiondf.iloc[confusiondf.shape[0] - 1, i] = fp
        # end of j
    # end of i
    
    # get the rest of the confusion matrix values
    confusiondf.loc[confusiondf.shape[0], 'Patients'] = 'TN' # - 3
    confusiondf.loc[confusiondf.shape[0], 'Patients'] = 'FN' # - 2
    confusiondf.loc[confusiondf.shape[0], 'Patients'] = 'TP-FP' # - 1
    
    for i in range(1, confusiondf.shape[1]):
        # TP - FP
        confusiondf.iloc[confusiondf.shape[0] - 1, i] = confusiondf.iloc[confusiondf.shape[0] - 5, i] - confusiondf.iloc[confusiondf.shape[0] - 4, i]
        # TN
        confusiondf.iloc[confusiondf.shape[0] - 2, i] = NCcount - confusiondf.iloc[confusiondf.shape[0] - 4, i]
        # FN
        confusiondf.iloc[confusiondf.shape[0] - 3, i] = Ccount - confusiondf.iloc[confusiondf.shape[0] - 4, i]

    return confusiondf


In [1166]:
# given a dataframe with a TP-FP row we make a decision tree based off those top values
def descisiontree(decisiondf1, decisiondf2):
    # merge dataframes together to get a df with the 108 patients and all columns
    decisiondf = pd.merge(decisiondf1['Patients'], decisiondf2['Patients'], on=['Patients', 'Patients'], how='outer')
    decisiondf = pd.merge(decisiondf, df[0:162], on=['Patients','Patients'], how='left')
    del decisiondf['sum_row']
    
    # get confusion matrix values
    decisiondf = setconfusion(decisiondf)
    
    # get the top TP-FP
    top = decisiondf.iloc[decisiondf.shape[0] - 1, 1:1410].sort_values(ascending=False).index
    
    # root node to section off the df
    root = top[0]

    # FOR INIT SPLITTING
    # make a list of patients
    list1C = []
    list1NC = []
    for i in range(0,decisiondf.shape[0] - 5):
        if decisiondf.at[i, root] == 1: # the the patient has the feature
            list1C.append(decisiondf.iat[i, 0])
        else: # patient doesnt have the feature
            list1NC.append(decisiondf.iat[i, 0])
    # end of i

    # now we take those 2 lists and make 2 new dataframes and split this 1 more time for the 3 total features
    group1 = pd.DataFrame(list1C, columns=['Patients'])
    group1 = pd.merge(group1, decisiondf,on=['Patients', 'Patients'], how='left')
    group2 = pd.DataFrame(list1NC, columns=['Patients'])
    group2 = pd.merge(group2, decisiondf,on=['Patients', 'Patients'], how='left')

    # get confusion matrix for those, and delete the already used feature so it doesnt duplicate
    del group1[root]
    del group2[root]
    group1 = setconfusion(group1)
    group2 = setconfusion(group2)
    
    # get nodes
    topC = group1.iloc[group1.shape[0] - 1, 1:1410].sort_values(ascending=False).index
    topNC = group2.iloc[group2.shape[0] - 1, 1:1410].sort_values(ascending=False).index
    nodeC = topC[0]
    nodeNC = topNC[0]
    
    retlist = [root, nodeC, nodeNC]
    return retlist 


def evaluator(list, evaldf):
    # this will take the list and evaluate a dataframe against it
    # the list will have the root node at pos 0, left node is pos 1, right node is pos 2
    
    for i in range(0, evaldf.shape[0]):
        if evaldf.at[i, list[0]] == 1: # it has the root node
            # we check it on the left node
            if evaldf.loc[i, list[1]] == 1:
                evaldf.at[i,'Evaluator'] = 'C'
            else: 
                evaldf.at[i,'Evaluator'] = 'NC'
        else: # doesnt have root node, check on right node pos 2
            if evaldf.at[i, list[2]] == 1:
                evaldf.at[i,'Evaluator'] = 'C'
            else:
                evaldf.at[i,'Evaluator'] = 'NC'
    # returns the dataframe with a new column for if the patient is C or NC
    return evaldf

# print("Classifier Stats:\nTP: ", tp, ' fp: ', fp, ' tn: ', tn, ' fn: ', fn, ' Accuracy: ', accuracy, ' Sensitivity: ', sensitivity)
# print('Specificity', specificity, ' Precision: ', precision, ' Miss Rate: ', missrate, '\nFalse Discovery Rate: ', fdr, ' False Ommision Rate: ', falseomrate)
# function that gets accuracy, sense and stuff
def compare(CDF):
    # compare the evaluator against actual then return that to a column and then make a confusion matrix off that
    # then make all the extra calculations needed for sense, accuracy, etc
    Ctotal, NCtotal, tp, tn = 0, 0, 0, 0
    for i in range(0, CDF.shape[0]):
        # print(x)
        x = CDF.iat[i, 0]
        y = CDF.iat[i, CDF.shape[1] - 1] 
        if x.find('C') == 0:
            Ctotal += 1
        if x.find('NC') == 0:
            NCtotal += 1
        if x.find('C') == 0 and y.find('C') == 0:
            tp += 1
        if x.find('NC') == 0 and y.find('NC') == 0:
            tn += 1
    # end of i
    # print(NCtotal, Ctotal)
    fp = NCtotal - tn
    fn = Ctotal - tp
    
    # inits
    accuracy, sensitivity, specificity, precision, missrate, fdr, falseomrate = 0, 0, 0, 0, 0, 0, 0
    
    # calculate the extra stuff
    # (tp + tn) / p + n
    accuracy = ((tp + tn) / CDF.shape[0])
    # tp / (tp + fn)
    if tp + fn != 0:
        sensitivity = tp / (tp + fn)
    else:
         sensitivity = tp / .0001
    # tn / (fp + tn)
    if fp + tn != 0:
        specificity = tn / (fp + tn)
    else:
            specificity = tn / .0001
    # tp / (tp + fp)
    if tp + fp != 0:
        precision = tp / (tp + fp)
    else:
        precision = tp / .0001
    # fn / (tp + fn)
    if tp + fn != 0:
        missrate = fn / (tp + fn)
    else:
        missrate = fn / .0001
    # fp / (tp + fp)
    if tp + fp != 0:
        fdr = fp / (tp + fp)
    else:
        fdr = fp / .0001
    # fn / (fn + tn)
    if tn + fn != 0:
        falseomrate = fn / (fn + tn)
    else:
        falseomrate = fn / .0001
    

    compresult = pd.DataFrame(columns=['Values'])
    compresult.loc['TP'] = tp
    compresult.loc['FP'] = fp
    compresult.loc['TN'] = tn
    compresult.loc['FN'] = fn
    compresult.loc['Accuracy'] = accuracy
    compresult.loc['Sensitivity'] = sensitivity
    compresult.loc['Specificity'] = specificity
    compresult.loc['Missrate'] = missrate
    compresult.loc['Precision'] = precision
    compresult.loc['FDR'] = fdr
    compresult.loc['FOR'] = falseomrate
    
    return compresult

In [97]:
################################################## Groups 1 and 2 ##################################################
# dectree will have a list of 3 features
dectree = descisiontree(onethird, twothird)

# send the list to the evaluator 
threethird = evaluator(dectree, threethird)

# get the calculated values/confusion matrix
hw6result1 = compare(threethird)

# output
hw6result1

,Values
TP,4
FP,0
TN,34
FN,16
Accuracy,0.703704
Sensitivity,0.2
Specificity,1.0
Missrate,0.8
Precision,1.0
FDR,0.0


In [98]:
################################################## Groups 1 and 3 ##################################################
# dectree will have a list of 3 features
dectree2 = descisiontree(onethird, threethird)

# send the list to the evaluator 
twothird = evaluator(dectree2, twothird)

# get the calculated values/confusion matrix
hw6result2 = compare(twothird)

# output
hw6result2


,Values
TP,2
FP,2
TN,36
FN,14
Accuracy,0.703704
Sensitivity,0.125
Specificity,0.947368
Missrate,0.875
Precision,0.5
FDR,0.5


In [99]:
################################################## Groups 2 and 3 ##################################################
# dectree will have a list of 3 features
dectree3 = descisiontree(twothird, threethird)

# send the list to the evaluator 
onethird = evaluator(dectree3, onethird)

# get the calculated values/confusion matrix
hw6result3 = compare(onethird)

# output
hw6result3

,Values
TP,5
FP,0
TN,34
FN,15
Accuracy,0.722222
Sensitivity,0.25
Specificity,1.0
Missrate,0.75
Precision,1.0
FDR,0.0


In [109]:
# get the avg of the results
hw6resultavg = pd.DataFrame(columns=['Values'])
namelist = ['TP','FP','TN','FN','accuracy','sensitivity','specificity','missrate','precision','FDR','FOR']
for i in range(0, hw6result1.shape[0]):
    x = (hw6result1.iat[i, 0] + hw6result2.iat[i, 0] + hw6result3.iat[i, 0]) / 3
    hw6resultavg.loc[namelist[i]] = x

hw6resultavg

,Values
TP,3.666667
FP,0.666667
TN,34.666667
FN,15.000000
accuracy,0.709877
sensitivity,0.191667
specificity,0.982456
missrate,0.808333
precision,0.833333
FDR,0.166667


In [ ]:
############################################ HW 7 ##########################################################################
# t is a feature in a decision tree
# s is a patient that is gonna be split on node t in the tree
hw7df = df.iloc[0:162, 0:1411]
def computeQST(phidf):
    phidf = phidf.append(pd.Series(), ignore_index=True) # 162, phi
    phidf.loc[162,'Patients'] = 'PHI'
    phidf = phidf.append(pd.Series(), ignore_index=True) # 163, Q(s|t)
    phidf.loc[163,'Patients'] = 'Q(s|t)'
    phidf = phidf.append(pd.Series(), ignore_index=True) # 164, 2PLPR
    phidf.loc[164,'Patients'] = '2PLPR'
    phidf = phidf.append(pd.Series(), ignore_index=True) # 165, P(NC | tR)
    phidf.loc[165,'Patients'] = 'P(NC | tR)'    
    phidf = phidf.append(pd.Series(), ignore_index=True) # 166, P(C | tR)
    phidf.loc[166,'Patients'] = 'P(C | tR)' 
    phidf = phidf.append(pd.Series(), ignore_index=True) # 167, P(NC | tL)
    phidf.loc[167,'Patients'] = 'P(NC | tL)' 
    phidf = phidf.append(pd.Series(), ignore_index=True) # 168, P(C | tL)
    phidf.loc[168,'Patients'] = 'P(C | tL)' 
    phidf = phidf.append(pd.Series(), ignore_index=True) # 169, PR
    phidf.loc[169,'Patients'] = 'PR' 
    phidf = phidf.append(pd.Series(), ignore_index=True) # 170, PL
    phidf.loc[170,'Patients'] = 'PL'
    phidf = phidf.append(pd.Series(), ignore_index=True) # 171, n(tR, NC)
    phidf.loc[171,'Patients'] = 'n(tR, NC)' 
    phidf = phidf.append(pd.Series(), ignore_index=True) # 172, n(tR, C)
    phidf.loc[172,'Patients'] = 'n(tR, C)' 
    phidf = phidf.append(pd.Series(), ignore_index=True) # 173, n(tL, NC)
    phidf.loc[173,'Patients'] = 'n(tL, NC)' 
    phidf = phidf.append(pd.Series(), ignore_index=True) # 174, n(tL, C)
    phidf.loc[174,'Patients'] = 'n(tL, C)' 
    phidf = phidf.append(pd.Series(), ignore_index=True) # 175, n(tR)
    phidf.loc[175,'Patients'] = 'n(tR)' 
    phidf = phidf.append(pd.Series(), ignore_index=True) # 176, n(tL)
    phidf.loc[176,'Patients'] = 'n(tL)' 
    
    for i in range(1, phidf.shape[1]): 
        # inits
        Lsamplecount = 0
        Rsamplecount = 0
        LCcount = 0
        LNCcount = 0
        RCcount = 0
        RNCcount = 0
        left_split = []
        right_split = []
        for j in range(0, phidf.shape[0]):
            if phidf.iloc[j, i] == 1:
                left_split.append(phidf.iloc[j,0]) # gets the patient
            elif phidf.iloc[j,i] == 0:
                right_split.append(phidf.iloc[j,0]) # gets the patient
        # calculate goodness: 2P_l * P_r * Q(s|t)
        # Q(s|t): |P(C | tL) - P(C | tR)| + |P(NC | tL) - P(NC | tR)| 
        # P(C|tL): n(tL, C) / n(tL)
        # n(tL, C): number of class ‘C’ samples at tL
            
        
        # get number of Cancer vs NC in left
        for k in left_split:
            if k.find('C') == 0:
                LCcount += 1
            else:
                LNCcount += 1
        # end of k
        # get number of Cancer vs NC in Right
        for k in right_split:
            if k.find('C') == 0:
                RCcount += 1
            else:
                RNCcount += 1
        # end of k
        # amount of samples in the split
        Lsamplecount = len(left_split) 
        Rsamplecount = len(right_split)
        
        # check for 0
        if Lsamplecount == 0:
            Lsamplecount = .0001
        if Rsamplecount == 0:
            Rsamplecount = .0001
        
        # piece together the formula for Q(s|t)
        pcl = LCcount / Lsamplecount  # P(C | tL)
        pcr = RCcount / Rsamplecount # P(C | tR)
        qp1 = abs(pcl - pcr) # part 1 of the formula : |P(C | tL) - P(C | tR)|
        pncl = LNCcount / Lsamplecount # P(NC | tL)
        pncr = RNCcount / Rsamplecount # P(NC | tR)
        qp2 = abs(pncl - pncr) # part 2 of the formula : |P(NC | tL) - P(NC | tR)| 
        qst = qp1 + qp2  # whole formula for Q(s|t)
        
        # piecing for phi itself ph(s,t) = 2PLPR * Q(s|t)
        # PL = n(tL) / n(t)
	    # PR = n(tR) / n(t)
        # n(tr) is num of samples in right tree
        # n(t) is 162
        pl = Lsamplecount / 162
        pr = Rsamplecount / 162
        phi = 2 * pl * pr * qst

        # fill in stuff
        phidf.iloc[phidf.shape[0] - 15, i] = phi
        phidf.iloc[phidf.shape[0] - 14, i] = qst
        phidf.iloc[phidf.shape[0] - 13, i] = 2 * pl * pr
        phidf.iloc[phidf.shape[0] - 12, i] = pncr
        phidf.iloc[phidf.shape[0] - 11, i] = pcr
        phidf.iloc[phidf.shape[0] - 10, i] = pncl
        phidf.iloc[phidf.shape[0] - 9, i] = pcl
        phidf.iloc[phidf.shape[0] - 8, i] = pr
        phidf.iloc[phidf.shape[0] - 7, i] = pl
        phidf.iloc[phidf.shape[0] - 6, i] = RNCcount
        phidf.iloc[phidf.shape[0] - 5, i] = RCcount
        phidf.iloc[phidf.shape[0] - 4, i] = LNCcount
        phidf.iloc[phidf.shape[0] - 3, i] = LCcount
        phidf.iloc[phidf.shape[0] - 2, i] = Rsamplecount
        phidf.iloc[phidf.shape[0] - 1, i] = Lsamplecount

    # end of i
    return phidf
# end of function

hw7df = computeQST(hw7df)

hw7df

/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/2881027531.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  phidf = phidf.append(pd.Series(), ignore_index=True) # 162, phi
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/2881027531.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  phidf = phidf.append(pd.Series(), ignore_index=True) # 163, Q(s|t)
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/2881027531.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  phidf = phidf.append(pd.Series(), ignore_index=True) # 164, 2PLPR
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipy

,Patients,FARP1_GRCh37_13:99092237-99092237_Frame-Shift-Del_DEL_G-G--,QKI_GRCh37_6:163987695-163987695_Intron_DEL_T-T--,NHLRC2_GRCh37_10:115662308-115662308_Frame-Shift-Del_DEL_A-A--,PDZD8_GRCh37_10:119042898-119042898_Frame-Shift-Del_DEL_T-T--,EBF3_GRCh37_10:131666125-131666125_Frame-Shift-Del_DEL_C-C--,MASTL_GRCh37_10:27459272-27459272_Frame-Shift-Del_DEL_A-A--,PTCHD3_GRCh37_10:27687763-27687763_Frame-Shift-Del_DEL_A-A--,PFKP_GRCh37_10:3151612-3151612_Silent_SNP_C-C-T,CREM_GRCh37_10:35456559-35456559_Intron_DEL_T-T--,...,PHF2_GRCh37_9:96422621-96422621_Missense-Mutation_SNP_A-A-G,PHF2_GRCh37_9:96422642-96422642_Missense-Mutation_SNP_C-C-A,USP51_GRCh37_X:55513209-55513209_3'UTR_DEL_T-T--,CDH23_GRCh37_10:73405660-73405660_Missense-Mutation_SNP_G-G-A,SLC26A11_GRCh37_17:78195370-78195370_Missense-Mutation_SNP_C-C-T,MME_GRCh37_3:154866382-154866382_Missense-Mutation_SNP_A-A-T_A-A-C,CADPS_GRCh37_3:62459960-62459960_Missense-Mutation_SNP_G-G-T,SCN4A_GRCh37_17:62049998-62049998_Silent_SNP_G-G-A,FPR2_GRCh37_19:52272624-52272624_Missense-Mutation_SNP_G-G-A,OLFM3_GRCh37_1:102302454-102302454_Missense-Mutation_SNP_A-A-C_A-A-G
0,C1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NC1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,C3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NC2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,"n(tR, C)",54.0,53.0,54.0,54.0,52.0,55.0,53.0,54.0,54.0,...,54.0,54.0,55.0,55.0,55.0,55.0,56.0,55.0,56.0,54.0
173,"n(tL, NC)",0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,0.0
174,"n(tL, C)",2.0,3.0,2.0,2.0,4.0,1.0,3.0,2.0,2.0,...,2.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0
175,n(tR),160.0,158.0,160.0,160.0,158.0,160.0,159.0,160.0,160.0,...,160.0,160.0,160.0,160.0,160.0,160.0,160.0,160.0,160.0,160.0


In [102]:
# rename to be able to sort
hw7df.iloc[162,0] = 9999999
# sort and output, note that 162 is PHI
hw7df.iloc[162:177,:].sort_values(by=hw7df.shape[0]-15, axis=1, ascending=False)


,Patients,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--,RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--,ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--,PGM5_GRCh37_9:70993145-70993145_Missense-Mutation_SNP_A-A-G,UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--,LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del_DEL_T-T--,RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--,KDM1B_GRCh37_6:18222307-18222307_3'UTR_DEL_A-A--,...,ABCA13_GRCh37_7:48312083-48312083_Missense-Mutation_SNP_A-A-C,MED1_GRCh37_17:37564657-37564659_In-Frame-Del_DEL_AGG-AGG--,UNC13A_GRCh37_19:17756598-17756598_Silent_SNP_G-G-A,EIF3E_GRCh37_8:109214102-109214102_3'UTR_SNP_C-C-T_C-C-G,TP53_GRCh37_17:7578265-7578265_Missense-Mutation_SNP_A-A-G_A-A-T,KRAS_GRCh37_12:25398284-25398284_Missense-Mutation_SNP_C-C-T_C-C-A,ZNF124_GRCh37_1:247319909-247319909_Frame-Shift-Del_DEL_T-T--,TP53_GRCh37_17:7577121-7577121_Missense-Mutation_SNP_G-G-A_G-G-T,TRIM49C_GRCh37_11:89774338-89774338_Missense-Mutation_SNP_A-A-C,CD1E_GRCh37_1:158324361-158324361_Missense-Mutation_SNP_T-T-G
162,9999999,0.193873,0.169182,0.169182,0.169182,0.161561,0.153025,0.129249,0.128334,0.113093,...,0.007621,0.007621,0.007621,0.007621,0.007621,0.006706,0.000914,0.000914,0.000914,0.000914
163,Q(s|t),1.413333,1.233333,1.233333,1.233333,1.394737,1.208910,1.376623,1.013847,1.367742,...,0.312500,0.312500,0.312500,0.312500,0.312500,0.112102,0.025157,0.025157,0.025157,0.025157
164,2PLPR,0.137174,0.137174,0.137174,0.137174,0.115836,0.126581,0.093888,0.126581,0.082686,...,0.024387,0.024387,0.024387,0.024387,0.024387,0.059823,0.036351,0.036351,0.036351,0.036351
165,P(NC | tR),0.706667,0.700000,0.700000,0.700000,0.697368,0.695364,0.688312,0.688742,0.683871,...,0.656250,0.656250,0.656250,0.656250,0.656250,0.656051,0.654088,0.654088,0.654088,0.654088
166,P(C | tR),0.293333,0.300000,0.300000,0.300000,0.302632,0.304636,0.311688,0.311258,0.316129,...,0.343750,0.343750,0.343750,0.343750,0.343750,0.343949,0.345912,0.345912,0.345912,0.345912
167,P(NC | tL),0.000000,0.083333,0.083333,0.083333,0.000000,0.090909,0.000000,0.181818,0.000000,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.600000,0.666667,0.666667,0.666667,0.666667
168,P(C | tL),1.000000,0.916667,0.916667,0.916667,1.000000,0.909091,1.000000,0.818182,1.000000,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.400000,0.333333,0.333333,0.333333,0.333333
169,PR,0.925926,0.925926,0.925926,0.925926,0.938272,0.932099,0.950617,0.932099,0.956790,...,0.987654,0.987654,0.987654,0.987654,0.987654,0.969136,0.981481,0.981481,0.981481,0.981481
170,PL,0.074074,0.074074,0.074074,0.074074,0.061728,0.067901,0.049383,0.067901,0.043210,...,0.012346,0.012346,0.012346,0.012346,0.012346,0.030864,0.018519,0.018519,0.018519,0.018519
171,"n(tR, NC)",106.000000,105.000000,105.000000,105.000000,106.000000,105.000000,106.000000,104.000000,106.000000,...,105.000000,105.000000,105.000000,105.000000,105.000000,103.000000,104.000000,104.000000,104.000000,104.000000


In [103]:
# rename the col back
hw7df.iloc[162,0] = 'PHI'


In [104]:
############################################ HW 7 Part 2 ##########################################################################
def topphi(phidf):
    # make a row for phi
    phidf = phidf.append(pd.Series(), ignore_index=True) # phi
    phidf.loc[phidf.shape[0] - 1,'Patients'] = 'PHI'
    
    # loop through patients
    for i in range(1, phidf.shape[1]):
        Lsamplecount = 0
        Rsamplecount = 0
        LCcount = 0
        LNCcount = 0
        RCcount = 0
        RNCcount = 0
        left_split = []
        right_split = []
        for j in range(0, phidf.shape[0]):
            if phidf.iloc[j, i] == 1:
                left_split.append(phidf.iloc[j,0]) 
            elif phidf.iloc[j,i] == 0:
                right_split.append(phidf.iloc[j,0])
        
        # end of j
        # get number of Cancer vs NC in left
        for k in left_split:
            if k.find('C') == 0:
                LCcount += 1
            else:
                LNCcount += 1
        # end of k
        # get number of Cancer vs NC in Right
        for k in right_split:
            if k.find('C') == 0:
                RCcount += 1
            else:
                RNCcount += 1
        # end of k
        # amount of samples in the split
        Lsamplecount = len(left_split) 
        Rsamplecount = len(right_split)
        
        # check for 0
        if Lsamplecount == 0:
            Lsamplecount = .0001
        if Rsamplecount == 0:
            Rsamplecount = .0001
        
        # piece together the formula for Q(s|t)
        pcl = LCcount / Lsamplecount  # P(C | tL)
        pcr = RCcount / Rsamplecount # P(C | tR)
        qp1 = abs(pcl - pcr) # part 1 of the formula : |P(C | tL) - P(C | tR)|
        pncl = LNCcount / Lsamplecount # P(NC | tL)
        pncr = RNCcount / Rsamplecount # P(NC | tR)
        qp2 = abs(pncl - pncr) # part 2 of the formula : |P(NC | tL) - P(NC | tR)| 
        qst = qp1 + qp2  # whole formula for Q(s|t)
        
        # piecing for phi itself ph(s,t) = 2PLPR * Q(s|t)
        # PL = n(tL) / n(t)
	    # PR = n(tR) / n(t)
        # n(tr) is num of samples in right tree
        # n(t) is 162
        pl = Lsamplecount / 162
        pr = Rsamplecount / 162
        phi = 2 * pl * pr * qst
    
        phidf.iloc[phidf.shape[0] - 1, i] = phi
    # end of i
    return phidf
# end of function

# given two dfs with phi values make a 2 level decision tree
# returns a list of 3 nodes
def phidecisiontree(phidf1, phidf2):
    # merge together the dataframes
    phidecdf = pd.merge(phidf1['Patients'], phidf2['Patients'], on=['Patients','Patients'], how='outer')
    phidecdf = pd.merge(phidecdf, df[0:162],on=['Patients','Patients'], how='left')
    # remove sum row
    del phidecdf['sum_row']
    
    # get phi values
    phidecdf = topphi(phidecdf)
    
    print("Original Goodness:")
    print(phidecdf.iloc[phidecdf.shape[0] - 1, 1:1411].sort_values(ascending=False))
    
    # get top phi value
    top = phidecdf.iloc[phidecdf.shape[0] - 1, 1:1411].sort_values(ascending=False).index
    
    # get root node
    root = top[0]
    
    left_split = []
    right_split = []
    
    # split the root into trees
    for i in range(0, phidecdf.shape[0] - 1):
        if phidecdf.at[i, root] == 1: 
            left_split.append(phidecdf.iat[i, 0])
        else:
            right_split.append(phidecdf.iat[i, 0])        
    # end of i
    
    # go through left and right to find their split
    left = pd.DataFrame(left_split, columns=['Patients'])
    left = pd.merge(left, phidecdf,on=['Patients', 'Patients'], how='left')
    right = pd.DataFrame(right_split, columns=['Patients'])
    right = pd.merge(right, phidecdf,on=['Patients', 'Patients'], how='left')
    
    # delete root so we dont repeat
    del left[root]
    del right[root]
    
    # get phi values
    left = topphi(left)
    right = topphi(right)
    
    # get nodes
    topleft = left.iloc[left.shape[0] - 1, 1:1411].sort_values(ascending=False).index
    topright = right.iloc[right.shape[0] - 1, 1:1411].sort_values(ascending=False).index
    nodeleft = topleft[0]
    noderight = topright[0]
    
    # for output
    print("\nLeft Goodness: ")
    print(left.iloc[left.shape[0] - 1, 1:1411].sort_values(ascending=False))
    print("\nRight Goodness:")
    print(right.iloc[right.shape[0] - 1, 1:1411].sort_values(ascending=False))
    
    retlist = [root, nodeleft, noderight]
    
    return retlist
# end of function

def phiclassifer(list1, classdf):
    left_split = []
    right_split = []
    for i in range(0, classdf.shape[0] - 1):
        if classdf.at[i, list1[0]] == 1:  # if it has the root
            left_split.append(classdf.iat[i,0])
        else:
            right_split.append(classdf.iat[i,0])
    # end of i
    la1 = []
    la2 = []
    ra1 = []
    ra2 = []
    # star to classify
    for i in range(0, classdf.shape[0] - 1):
        if classdf.at[i, list1[1]] == 1: # it has feature A
            la1.append(classdf.iat[i, 0])
        elif classdf.at[i, list1[1]] == 0:
            la2.append(classdf.iat[i, 0])
        if classdf.at[i, list1[2]] == 1:
            ra1.append(classdf.iat[i, 0])
        elif classdf.at[i, list1[2]] == 0:
            ra2.append(classdf.iat[i, 0])
    # end of i
    la1Cc = 0
    la1NCc = 0
    la2Cc = 0
    la2NCc = 0
    ra1Cc = 0
    ra1NCc = 0
    ra2Cc = 0
    ra2NCc = 0
    for i in la1:
        if i.find('C') == 0:
            la1Cc += 1
        else:
            la1NCc += 1
    for i in la2:
        if i.find('C') == 0:
            la2Cc += 1
        else:
            la2NCc += 1
    for i in ra1:
        if i.find('C') == 0:
            ra1Cc += 1
        else:
            ra1NCc += 1
    for i in ra2:
        if i.find('C') == 0:
            ra2Cc += 1
        else:
            ra2NCc += 1
    
    for i in range(0, classdf.shape[0]):
        if classdf.at[i, list1[1]] == 1: # it has feature A
            if la1Cc > la1NCc:
                classdf.at[i, 'Evaluator'] = 'C'
            else:
                classdf.at[i, 'Evaluator'] = 'NC'
        elif classdf.at[i, list1[1]] == 0: # doesnt have feature A
            if la2Cc > la2NCc:
                classdf.at[i, 'Evaluator'] = 'C'
            else:
                classdf.at[i, 'Evaluator'] = 'NC'
        if classdf.at[i, list1[2]] == 1:
            if ra1Cc > ra1NCc:
                classdf.at[i, 'Evaluator'] = 'C'
            else:
                classdf.at[i, 'Evaluator'] = 'NC'
        elif classdf.at[i, list1[2]] == 0:
            if ra2Cc > ra2NCc:
                classdf.at[i, 'Evaluator'] = 'C'
            else:
                classdf.at[i, 'Evaluator'] = 'NC'
    # end of i
    return classdf
# end of function


In [110]:
# get the samples randomly mixed up
tmpdf3 = df.iloc[0:162, 0:1411].sample(n=162, replace=False)

# reset the indexes
tmpdf3.reset_index(drop=True)

# section it into 3 groups
hw7onethird = tmpdf3[0:54]
hw7twothird = tmpdf3[54:108]
hw7threethird = tmpdf3[108:162]

# reset the indexes on them
hw7onethird = hw7onethird.reset_index(drop=True)
hw7twothird = hw7twothird.reset_index(drop=True)
hw7threethird = hw7threethird.reset_index(drop=True)

In [111]:
################################################## Groups 1 and 3 ##################################################
# get decision tree
phidectree2 = phidecisiontree(hw7onethird, hw7threethird)

# classifier
hw7twothird = phiclassifer(phidectree2, hw7twothird)

# evaluate
hw7result2 = compare(hw7twothird)

hw7result2

/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/1211698872.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  phidf = phidf.append(pd.Series(), ignore_index=True) # phi


Original Goodness:
ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--     0.086572
UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--       0.064929
KIAA0195_GRCh37_17:73491063-73491063_Frame-Shift-Del_DEL_C-C--    0.064929
LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del_DEL_T-T--          0.064929
DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--        0.064929
                                                                    ...   
AXL_GRCh37_19:41744385-41744385_Frame-Shift-Del_DEL_C-C--         0.000001
MAPKAPK3_GRCh37_3:50655080-50655080_Frame-Shift-Del_DEL_G-G--     0.000001
SCN11A_GRCh37_3:38949441-38949441_Frame-Shift-Del_DEL_T-T--       0.000001
FGF22_GRCh37_19:643528-643528_Frame-Shift-Del_DEL_G-G--           0.000001
FARP1_GRCh37_13:99092237-99092237_Frame-Shift-Del_DEL_G-G--       0.000001
Name: 108, Length: 1410, dtype: object

Left Goodness: 
FARP1_GRCh37_13:99092237-99092237_Frame-Shift-Del_DEL_G-G--      0.0
ZDHHC23_GRCh37_3:113679540-1136

,Values
TP,5
FP,0
TN,35
FN,14
Accuracy,0.740741
Sensitivity,0.263158
Specificity,1.0
Missrate,0.736842
Precision,1.0
FDR,0.0


In [107]:
################################################## Groups 2 and 3 ##################################################
# get decision tree
phidectree1 = phidecisiontree(hw7twothird, hw7threethird)

# classifier
hw7onethird = phiclassifer(phidectree1, hw7onethird)

# evaluate
hw7result1 = compare(hw7onethird)

hw7result1

/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/1211698872.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  phidf = phidf.append(pd.Series(), ignore_index=True) # phi


Original Goodness:
DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--            0.087791
ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--         0.082305
RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--           0.076818
ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--         0.071331
UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--           0.071331
                                                                        ...   
GTF3C4_GRCh37_9:135554691-135554691_Frame-Shift-Del_DEL_A-A--         0.000001
RIMS2_GRCh37_8:105235897-105235897_Intron_DEL_G-G--                   0.000001
KRAS_GRCh37_12:25398285-25398285_Missense-Mutation_SNP_C-C-A_C-C-T         0.0
TP53_GRCh37_17:7578403-7578403_Missense-Mutation_SNP_C-C-T_C-C-A           0.0
CD1E_GRCh37_1:158324361-158324361_Missense-Mutation_SNP_T-T-G              0.0
Name: 108, Length: 1410, dtype: object

Left Goodness: 
OLFM3_GRCh37_1:102302454-102302454_Missense-Mutation_SNP

,Values
TP,3
FP,0
TN,34
FN,17
Accuracy,0.685185
Sensitivity,0.15
Specificity,1.0
Missrate,0.85
Precision,1.0
FDR,0.0


In [108]:
################################################## Groups 1 and 2 ##################################################
# get decision tree
phidectree3 = phidecisiontree(hw7onethird, hw7twothird)

# classifier
hw7threethird = phiclassifer(phidectree3, hw7threethird)

# evaluate
hw7result3 = compare(hw7threethird)

hw7result3

/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/1211698872.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  phidf = phidf.append(pd.Series(), ignore_index=True) # phi


Original Goodness:
DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--              0.098765
RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--             0.093278
UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--             0.087791
ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--           0.082305
ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--           0.076818
                                                                          ...   
ALG10_GRCh37_12:34179403-34179404_Frame-Shift-Ins_INS_----T             0.000001
PRB4_GRCh37_12:11461619-11461619_Nonsense-Mutation_SNP_C-C-A            0.000001
KIF20B_GRCh37_10:91514406-91514406_Frame-Shift-Del_DEL_T-T--            0.000001
OLFM3_GRCh37_1:102302454-102302454_Missense-Mutation_SNP_A-A-C_A-A-G    0.000001
TP53_GRCh37_17:7577094-7577094_Missense-Mutation_SNP_G-G-A                   0.0
Name: 108, Length: 1410, dtype: object


KeyboardInterrupt: 

In [ ]:
################################################## Groups Outputs ##################################################
print("Nodes for Group 1")
phidectree1

Nodes for Group 1


['PGM5_GRCh37_9:70993145-70993145_Missense-Mutation_SNP_A-A-G',
 'FARP1_GRCh37_13:99092237-99092237_Frame-Shift-Del_DEL_G-G--',
 'ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--']

In [ ]:
print("Nodes for Group 2")
phidectree2

Nodes for Group 2


['RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--',
 'OR52A5_GRCh37_11:5153679-5153679_Frame-Shift-Del_DEL_A-A--',
 "KDM1B_GRCh37_6:18222307-18222307_3'UTR_DEL_A-A--"]

In [ ]:
print("Nodes for Group 3")
phidectree3

Nodes for Group 3


['DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--',
 'OLFM3_GRCh37_1:102302454-102302454_Missense-Mutation_SNP_A-A-C_A-A-G',
 'ZC3H18_GRCh37_16:88691141-88691141_Frame-Shift-Del_DEL_C-C--']

In [ ]:
hw7p2resultavg = pd.DataFrame(columns=['Values'])
namelist = ['TP','FP','TN','FN','accuracy','sensitivity','specificity','missrate','precision','FDR','FOR']
for i in range(0, hw6result1.shape[0]):
    x = (hw7result1.iat[i, 0] + hw7result2.iat[i, 0] + hw7result3.iat[i, 0]) / 3
    hw7p2resultavg.loc[namelist[i]] = x

hw7p2resultavg

,Values
TP,2.666667
FP,0.333333
TN,35.000000
FN,16.000000
accuracy,0.697531
sensitivity,0.151416
specificity,0.990741
missrate,0.848584
precision,0.611111
FDR,0.055556


In [137]:
############################################ HW 8 ##########################################################################
# get a df for this hw
hw8df = df.iloc[0:162, 0:1411]

# get the samples randomly mixed up
tmpdf3 = df.iloc[0:162, 0:1411].sample(n=162, replace=False)

# reset the indexes
tmpdf3.reset_index(drop=True)

# section it into 3 groups
hw8onethird = tmpdf3[0:54]
hw8twothird = tmpdf3[54:108]
hw8threethird = tmpdf3[108:162]

# reset the indexes on them
hw8onethird = hw8onethird.reset_index(drop=True)
hw8twothird = hw8twothird.reset_index(drop=True)
hw8threethird = hw8threethird.reset_index(drop=True)

In [138]:
# will return a df with a row of entropy values at .shape[0] - 1
def make_gainp1(gaindf):
    # make the rows we ne need
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 162
    gaindf.loc[162,'Patients'] = 'Gain'
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 163
    gaindf.loc[163,'Patients'] = 'H(t)'
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 164
    gaindf.loc[164,'Patients'] = 'H(s,t)'
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 165
    gaindf.loc[165,'Patients'] = 'Pr'    
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 166
    gaindf.loc[166,'Patients'] = 'Pl' 
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 167
    gaindf.loc[167,'Patients'] = 'n(tR, NC)' 
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 168
    gaindf.loc[168,'Patients'] = 'n(tR, C)' 
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 169
    gaindf.loc[169,'Patients'] = 'n(tL, NC)' 
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 170
    gaindf.loc[170,'Patients'] = 'n(tL, C)'
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 171
    gaindf.loc[171,'Patients'] = 'n(tR)' 
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 172
    gaindf.loc[172,'Patients'] = 'n(tL)' 
    
    # go through the mutations
    for i in range(1, gaindf.shape[1]):
        # inits
        right_tree_Camount, right_tree_NCamount, left_tree_Camount, left_tree_NCamount = 0, 0, 0, 0
        right_tree = []
        left_tree = []
        
        # go through patients, do minus 11 so we dont go through the new rows
        for j in range(0, gaindf.shape[0] - 11):
            if gaindf.iloc[j, i] == 1:
                left_tree.append(gaindf.iloc[j,0]) # gets the patients
            else: # it == 0
                right_tree.append(gaindf.iloc[j,0]) # gets the patients
        # end of j
        
        # make h(t):
        pct = 56/162
        pnct = 106/162
        ht = -1 * ((pct * math.log(pct, 2)) + (pnct * math.log(pnct, 2)))
        
        # h(tl)
        # get c in the left tree
        for k in left_tree:
            if k.find('C') == 0:
                left_tree_Camount += 1 # get the amount of c
            else:
                left_tree_NCamount += 1 # get the amount of nc
        # end of k

        # h(tr)
        for k in right_tree:
            if k.find('C') == 0:
                right_tree_Camount += 1 # get the amount of c
            else:
                right_tree_NCamount += 1 # get the amount of nc
        # end of k
        
        # make h(s,t) = PLH(tL) + PRH(tR)
        #  pl = n(tl) / 162
        #  n(tl) = samples in left node len(left_tree)
        pctcl = left_tree_Camount / len(left_tree)
        pctncl = left_tree_NCamount / len(left_tree)
        pctrcr = right_tree_Camount / len(right_tree)
        pctrncr = right_tree_NCamount / len(right_tree)
        if pctcl == 0:
            pctcl = .0001
        if pctncl == 0:
            pctncl = .0001
        if pctrcr == 0:
            pctrcr = .0001
        if pctrncr == 0:
            pctrncr = .0001

        htl = -1 * ((pctcl * math.log(pctcl, 2)) + (pctncl * math.log(pctncl, 2)))
        htr = -1 * ((pctrcr * math.log(pctrcr, 2)) + (pctrncr * math.log(pctrncr, 2)))
        
        pl = len(left_tree) / 162
        pr = len(right_tree) / 162
        
        hst = (pl * htl) + (pr * htr)
        
        # make gain = h(t) - h(s,t)
        gain = ht - hst
        
        # load everything into the dataframe
        gaindf.iloc[gaindf.shape[0] - 11, i] = gain
        gaindf.iloc[gaindf.shape[0] - 10, i] = ht
        gaindf.iloc[gaindf.shape[0] - 9, i] = hst
        gaindf.iloc[gaindf.shape[0] - 8, i] = pr
        gaindf.iloc[gaindf.shape[0] - 7, i] = pl
        gaindf.iloc[gaindf.shape[0] - 6, i] = right_tree_NCamount
        gaindf.iloc[gaindf.shape[0] - 5, i] = right_tree_Camount
        gaindf.iloc[gaindf.shape[0] - 4, i] = left_tree_NCamount
        gaindf.iloc[gaindf.shape[0] - 3, i] = left_tree_Camount
        gaindf.iloc[gaindf.shape[0] - 2, i] = len(right_tree)
        gaindf.iloc[gaindf.shape[0] - 1, i] = len(left_tree)
        
    # end of i loop
    
    return gaindf
# end of function
 	

In [139]:
# sort by gain and give the top columns
hw8df = make_gainp1(hw8df)

# make gain title a num so we can sort
hw8df.iloc[162, 0] = 99999999

# sort
hw8df.iloc[162:177,:].sort_values(by=hw8df.shape[0]-11,axis=1,ascending=False)

#rename back
hw8df.iloc[162, 0] = 'Gain'

/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/597587889.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 162
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/597587889.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 163
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/597587889.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 164
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/59758

,Patients,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--,PGM5_GRCh37_9:70993145-70993145_Missense-Mutation_SNP_A-A-G,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--,ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--,RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--,LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del_DEL_T-T--,UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--,KIAA0195_GRCh37_17:73491063-73491063_Frame-Shift-Del_DEL_C-C--,UPF3A_GRCh37_13:115057211-115057211_Frame-Shift-Del_DEL_A-A--,...,HNRNPK_GRCh37_9:86585140-86585140_Missense-Mutation_SNP_C-C-A_C-C-T,ABCA13_GRCh37_7:48312083-48312083_Missense-Mutation_SNP_A-A-C,FGFR1OP_GRCh37_6:167417716-167417716_RNA_DEL_T-T--,TP53_GRCh37_17:7578406-7578406_Missense-Mutation_SNP_C-C-T,TP53_GRCh37_17:7577094-7577094_Missense-Mutation_SNP_G-G-A,KRAS_GRCh37_12:25398284-25398284_Missense-Mutation_SNP_C-C-T_C-C-A,ZNF124_GRCh37_1:247319909-247319909_Frame-Shift-Del_DEL_T-T--,TRIM49C_GRCh37_11:89774338-89774338_Missense-Mutation_SNP_A-A-C,TP53_GRCh37_17:7577121-7577121_Missense-Mutation_SNP_G-G-A_G-G-T,CD1E_GRCh37_1:158324361-158324361_Missense-Mutation_SNP_T-T-G
162,99999999,0.121729,0.100182,0.083487,0.083487,0.083487,0.079174,0.073644,0.068864,0.068864,...,0.000903,0.000903,0.000903,0.000779,0.000779,0.000293,0.000009,0.000009,0.000009,0.000009
163,H(t),0.930150,0.930150,0.930150,0.930150,0.930150,0.930150,0.930150,0.930150,0.930150,...,0.930150,0.930150,0.930150,0.930150,0.930150,0.930150,0.930150,0.930150,0.930150,0.930150
164,"H(s,t)",0.808421,0.829968,0.846663,0.846663,0.846663,0.850975,0.856506,0.861285,0.861285,...,0.929246,0.929246,0.929246,0.929371,0.929371,0.929856,0.930141,0.930141,0.930141,0.930141
165,Pr,0.925926,0.938272,0.925926,0.925926,0.925926,0.950617,0.932099,0.956790,0.956790,...,0.987654,0.987654,0.987654,0.975309,0.975309,0.969136,0.981481,0.981481,0.981481,0.981481
166,Pl,0.074074,0.061728,0.074074,0.074074,0.074074,0.049383,0.067901,0.043210,0.043210,...,0.012346,0.012346,0.012346,0.024691,0.024691,0.030864,0.018519,0.018519,0.018519,0.018519
167,"n(tR, NC)",106.000000,106.000000,105.000000,105.000000,105.000000,106.000000,105.000000,106.000000,106.000000,...,105.000000,105.000000,105.000000,103.000000,103.000000,103.000000,104.000000,104.000000,104.000000,104.000000
168,"n(tR, C)",44.000000,46.000000,45.000000,45.000000,45.000000,48.000000,46.000000,49.000000,49.000000,...,55.000000,55.000000,55.000000,55.000000,55.000000,54.000000,55.000000,55.000000,55.000000,55.000000
169,"n(tL, NC)",0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,3.000000,3.000000,3.000000,2.000000,2.000000,2.000000,2.000000
170,"n(tL, C)",12.000000,10.000000,11.000000,11.000000,11.000000,8.000000,10.000000,7.000000,7.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000
171,n(tR),150.000000,152.000000,150.000000,150.000000,150.000000,154.000000,151.000000,155.000000,155.000000,...,160.000000,160.000000,160.000000,158.000000,158.000000,157.000000,159.000000,159.000000,159.000000,159.000000


In [147]:
######################################### HW 8 Part 2 ####################################################
# reset the dataframe dont need the values from b4
hw8df = df.iloc[0:162, 0:1411]

In [382]:
# makes a row for gain values given an df
def topgain(gaindf):
    # make a row for gain
    gaindf = gaindf.append(pd.Series(), ignore_index=True)
    gaindf.loc[gaindf.shape[0] - 1, 'Patients'] = 'Gain'
    
    totalC, totalNC = 0,0
    # go through the dataframe and see how many C and NC there are for H(t)
    # go through and get how many c and NC there are in the total df
    for i in gaindf.iloc[0:gaindf.shape[0]-1,0]:
        if i.find('C') == 0:
            totalC += 1
        else:
            totalNC += 1
    # end of i 
    # make h(t):
    pct = totalC / (gaindf.shape[0] - 1)
    pnct = totalNC / (gaindf.shape[0] - 1)
    if pct == 0:
        pct = .0001
    if pnct == 0:
        pnct = .0001
    ht = -1 * ((pct * math.log(pct, 2)) + (pnct * math.log(pnct, 2)))
        
    # go through the mutations
    for i in range(1, gaindf.shape[1]):
        # inits
        right_tree_Camount, right_tree_NCamount, left_tree_Camount, left_tree_NCamount = 0, 0, 0, 0
        right_tree = []
        left_tree = []
        
        for j in range(0, gaindf.shape[0] - 2):
            if gaindf.iloc[j, i] == 1:
                left_tree.append(gaindf.iloc[j,0]) # gets the patients
            else: # it == 0
                right_tree.append(gaindf.iloc[j,0]) # gets the patients
        # end of j
        
        # h(tl)
        # get c in the left tree
        for k in left_tree:
            if k.find('C') == 0:
                left_tree_Camount += 1 # get the amount of c
            else:
                left_tree_NCamount += 1 # get the amount of nc
        # end of k

        # h(tr)
        for k in right_tree:
            if k.find('C') == 0:
                right_tree_Camount += 1 # get the amount of c
            else:
                right_tree_NCamount += 1 # get the amount of nc
        # end of k

        # make h(s,t) = PLH(tL) + PRH(tR)
        #  pl = n(tl) / 162
        #  n(tl) = samples in left node len(left_tree)
        pctcl, pctncl, pctrcr, pctrncr = .0001, .0001, .0001, .0001
        if len(left_tree) != 0:
            pctcl = left_tree_Camount / len(left_tree)
            pctncl = left_tree_NCamount / len(left_tree)
        if len(right_tree) != 0:
            pctrcr = right_tree_Camount / len(right_tree)
            pctrncr = right_tree_NCamount / len(right_tree)
            
        if pctcl == 0:
            pctcl = .0001
        if pctncl == 0:
            pctncl = .0001
        if pctrcr == 0:
            pctrcr = .0001
        if pctrncr == 0:
            pctrncr = .0001

        htl = -1 * ((pctcl * math.log(pctcl, 2)) + (pctncl * math.log(pctncl, 2)))
        htr = -1 * ((pctrcr * math.log(pctrcr, 2)) + (pctrncr * math.log(pctrncr, 2)))
        
        pl = len(left_tree) / (gaindf.shape[0] - 1)
        pr = len(right_tree) / (gaindf.shape[0] - 1)
        
        hst = (pl * htl) + (pr * htr)
        
        # make gain = h(t) - h(s,t)
        gain = ht - hst
        
        # load everything into the dataframe
        gaindf.iloc[gaindf.shape[0] - 1, i] = gain
        
    # end of i loop
    
    return gaindf
# end of function

# given 2 dataframes with merge them, get the gain values. Return list of 3 nodes 
def gaindecisiontree(gaindf1, gaindf2):
    # merge df together
    gaindf = pd.merge(gaindf1['Patients'], gaindf2['Patients'], on=['Patients','Patients'], how='outer')
    gaindf = pd.merge(gaindf,df[0:162], on=['Patients','Patients'], how='left')
    
    # for output  
    outputgaindf = gaindf
    
    # remove useless row
    del gaindf['sum_row']
    
    # get gain values
    gaindf = topgain(gaindf)

    # get the top gain 
    top = gaindf.iloc[gaindf.shape[0] - 1, 1:1411].sort_values(ascending=False).index
    
    # get root mutation
    root = top[0]

    # inits
    left_split = []
    right_split = []
    # split patients
    for i in range(0, gaindf.shape[0] - 1):
        if gaindf.at[i, root] == 1: 
            left_split.append(gaindf.iat[i, 0])
        else:
            right_split.append(gaindf.iat[i, 0]) 
    # end of i
    
    # go through left and right to find their split
    left = pd.DataFrame(left_split, columns=['Patients'])
    left = pd.merge(left, gaindf,on=['Patients', 'Patients'], how='left')
    right = pd.DataFrame(right_split, columns=['Patients'])
    right = pd.merge(right, gaindf,on=['Patients', 'Patients'], how='left')
    
    # delete root so we dont repeat
    del left[root]
    del right[root]
    
    # for output
    outputleft = left
    outputright = right
    
    # get gain values
    left = topgain(left)
    right = topgain(right)
    
    # get nodes
    topleft = left.iloc[left.shape[0] - 1, 1:1411].sort_values(ascending=False).index
    topright = right.iloc[right.shape[0] - 1, 1:1411].sort_values(ascending=False).index
    nodeleft = topleft[0]
    noderight = topright[0]
    
    # get tables for later
    global gainright
    gainright = outputright
    global gainleft 
    gainleft = outputleft
    global gainog
    gainog = outputgaindf

        
    retlist = [root, nodeleft, noderight]
    # Return list
    return retlist
#end of function

def makegaintable(gaindf):
    totalC, totalNC = 0,0
    # go through the dataframe and see how many C and NC there are for H(t)
    # go through and get how many c and NC there are in the total df
    for i in gaindf.iloc[0:gaindf.shape[0],0]:
        if i.find('C') == 0:
            totalC += 1
        else:
            totalNC += 1
    # end of i 
    # make h(t):
    pct = totalC / (gaindf.shape[0])
    pnct = totalNC / (gaindf.shape[0])
    if pct == 0:
        pct = .0001
    if pnct == 0:
        pnct = .0001
    ht = -1 * ((pct * math.log(pct, 2)) + (pnct * math.log(pnct, 2)))
    # make the rows we ne need
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 162
    gaindf.loc[gaindf.shape[0] - 1,'Patients'] = 'Gain'
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 163
    gaindf.loc[gaindf.shape[0] - 1,'Patients'] = 'H(t)'
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 164
    gaindf.loc[gaindf.shape[0] - 1,'Patients'] = 'H(s,t)'
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 165
    gaindf.loc[gaindf.shape[0] - 1,'Patients'] = 'Pr'    
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 166
    gaindf.loc[gaindf.shape[0] - 1,'Patients'] = 'Pl' 
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 167
    gaindf.loc[gaindf.shape[0] - 1,'Patients'] = 'n(tR, NC)' 
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 168
    gaindf.loc[gaindf.shape[0] - 1,'Patients'] = 'n(tR, C)' 
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 169
    gaindf.loc[gaindf.shape[0] - 1,'Patients'] = 'n(tL, NC)' 
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 170
    gaindf.loc[gaindf.shape[0] - 1,'Patients'] = 'n(tL, C)'
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 171
    gaindf.loc[gaindf.shape[0] - 1,'Patients'] = 'n(tR)' 
    gaindf = gaindf.append(pd.Series(), ignore_index=True) # 172
    gaindf.loc[gaindf.shape[0] - 1,'Patients'] = 'n(tL)' 
        
    # go through the mutations
    for i in range(1, gaindf.shape[1]):
        # inits
        right_tree_Camount, right_tree_NCamount, left_tree_Camount, left_tree_NCamount = 0, 0, 0, 0
        right_tree = []
        left_tree = []
        
        for j in range(0, gaindf.shape[0] - 11):
            if gaindf.iloc[j, i] == 1:
                left_tree.append(gaindf.iloc[j,0]) # gets the patients
            else: # it == 0
                right_tree.append(gaindf.iloc[j,0]) # gets the patients
        # end of j
        
        # h(tl)
        # get c in the left tree
        for k in left_tree:
            if k.find('C') == 0:
                left_tree_Camount += 1 # get the amount of c
            else:
                left_tree_NCamount += 1 # get the amount of nc
        # end of k

        # h(tr)
        for k in right_tree:
            if k.find('C') == 0:
                right_tree_Camount += 1 # get the amount of c
            else:
                right_tree_NCamount += 1 # get the amount of nc
        # end of k

        # make h(s,t) = PLH(tL) + PRH(tR)
        #  pl = n(tl) / 162
        #  n(tl) = samples in left node len(left_tree)
        pctcl, pctncl, pctrcr, pctrncr = .0001, .0001, .0001, .0001
        if len(left_tree) != 0:
            pctcl = left_tree_Camount / len(left_tree)
            pctncl = left_tree_NCamount / len(left_tree)
        if len(right_tree) != 0:
            pctrcr = right_tree_Camount / len(right_tree)
            pctrncr = right_tree_NCamount / len(right_tree)
            
        if pctcl == 0:
            pctcl = .0001
        if pctncl == 0:
            pctncl = .0001
        if pctrcr == 0:
            pctrcr = .0001
        if pctrncr == 0:
            pctrncr = .0001

        htl = -1 * ((pctcl * math.log(pctcl, 2)) + (pctncl * math.log(pctncl, 2)))
        htr = -1 * ((pctrcr * math.log(pctrcr, 2)) + (pctrncr * math.log(pctrncr, 2)))
        
        pl = len(left_tree) / (gaindf.shape[0] - 11)
        pr = len(right_tree) / (gaindf.shape[0] - 11)
        
        hst = (pl * htl) + (pr * htr)
        
        # make gain = h(t) - h(s,t)
        gain = ht - hst
        
        # load everything into the dataframe
        gaindf.iloc[gaindf.shape[0] - 11, i] = gain
        gaindf.iloc[gaindf.shape[0] - 10, i] = ht
        gaindf.iloc[gaindf.shape[0] - 9, i] = hst
        gaindf.iloc[gaindf.shape[0] - 8, i] = pr
        gaindf.iloc[gaindf.shape[0] - 7, i] = pl
        gaindf.iloc[gaindf.shape[0] - 6, i] = right_tree_NCamount
        gaindf.iloc[gaindf.shape[0] - 5, i] = right_tree_Camount
        gaindf.iloc[gaindf.shape[0] - 4, i] = left_tree_NCamount
        gaindf.iloc[gaindf.shape[0] - 3, i] = left_tree_Camount
        gaindf.iloc[gaindf.shape[0] - 2, i] = len(right_tree)
        gaindf.iloc[gaindf.shape[0] - 1, i] = len(left_tree)

    # end of i loop
    
    return gaindf
# end of function
 	


In [383]:
################################################## Groups 1 and 3 ##################################################
# get decision tree
gaindecisiontree1 = gaindecisiontree(hw8onethird, hw8threethird)

# classifier
hw8twothird = phiclassifer(gaindecisiontree1, hw8twothird)

# evaluate
hw8result2 = compare(hw8twothird)

# output results and the decision tree nodes
print("Nodes: ")
print(gaindecisiontree1)
hw8result2

/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True)


Nodes: 
['DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--', 'FARP1_GRCh37_13:99092237-99092237_Frame-Shift-Del_DEL_G-G--', 'PGM5_GRCh37_9:70993145-70993145_Missense-Mutation_SNP_A-A-G']


,Values
TP,2
FP,0
TN,40
FN,12
Accuracy,0.777778
Sensitivity,0.142857
Specificity,1.0
Missrate,0.857143
Precision,1.0
FDR,0.0


In [384]:
# output OG table with values
outputog = makegaintable(gainog)
outputog.iloc[108, 0] = 999999
outputog.iloc[108:,:].sort_values(by=outputog.shape[0]-11,axis=1,ascending=False)


/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:184: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 162
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:186: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 163
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:188: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 164
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42

,Patients,PGM5_GRCh37_9:70993145-70993145_Missense-Mutation_SNP_A-A-G,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--,RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--,LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del_DEL_T-T--,MVK_GRCh37_12:110019240-110019240_Frame-Shift-Del_DEL_C-C--,KIAA0195_GRCh37_17:73491063-73491063_Frame-Shift-Del_DEL_C-C--,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--,KDM1B_GRCh37_6:18222307-18222307_3'UTR_DEL_A-A--,NFASC_GRCh37_1:204924033-204924033_Frame-Shift-Del_DEL_C-C--,...,MYCBP2_GRCh37_13:77632491-77632491_Silent_SNP_G-G-A,ZNF7_GRCh37_8:146067177-146067178_Frame-Shift-Ins_INS_----A,RCBTB1_GRCh37_13:50140831-50140831_Frame-Shift-Del_DEL_T-T--,CSPP1_GRCh37_8:68076631-68076631_Frame-Shift-Del_DEL_A-A--,MYO9A_GRCh37_15:72190580-72190580_Frame-Shift-Del_DEL_A-A--,C8orf34_GRCh37_8:69445431-69445431_Intron_DEL_A-A--,RP11-252A24.2_GRCh37_16:74372644-74372644_RNA_SNP_A-A-G,SH3GLB2_GRCh37_9:131771452-131771452_Missense-Mutation_SNP_C-C-T,RAB11FIP2_GRCh37_10:119798647-119798647_Frame-Shift-Del_DEL_T-T--,HAP1_GRCh37_17:39890671-39890671_Silent_SNP_C-C-T
108,999999,0.107667,0.107667,0.084332,0.065593,0.065593,0.065593,0.065593,0.065593,0.052044,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
109,H(t),0.964079,0.964079,0.964079,0.964079,0.964079,0.964079,0.964079,0.964079,0.964079,...,0.964079,0.964079,0.964079,0.964079,0.964079,0.964079,0.964079,0.964079,0.964079,0.964079
110,"H(s,t)",0.856412,0.856412,0.879746,0.898485,0.898485,0.898485,0.898485,0.898485,0.912035,...,0.964079,0.964079,0.964079,0.964079,0.964079,0.964079,0.964079,0.964079,0.964079,0.964079
111,Pr,0.925926,0.925926,0.907407,0.953704,0.953704,0.953704,0.953704,0.953704,0.962963,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
112,Pl,0.074074,0.074074,0.092593,0.046296,0.046296,0.046296,0.046296,0.046296,0.037037,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
113,"n(tR, NC)",66.000000,66.000000,65.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,...,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000
114,"n(tR, C)",34.000000,34.000000,33.000000,37.000000,37.000000,37.000000,37.000000,37.000000,38.000000,...,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000
115,"n(tL, NC)",0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
116,"n(tL, C)",8.000000,8.000000,9.000000,5.000000,5.000000,5.000000,5.000000,5.000000,4.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
117,n(tR),100.000000,100.000000,98.000000,103.000000,103.000000,103.000000,103.000000,103.000000,104.000000,...,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000


In [385]:
# output left table with values
outputleft = makegaintable(gainleft)
outputleft.iloc[outputleft.shape[0]-11, 0] = 999999
outputleft.iloc[outputleft.shape[0]-11:,:].sort_values(by=outputleft.shape[0]-11,axis=1,ascending=False)


/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:184: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 162
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:186: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 163
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:188: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 164
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42

,Patients,ELOVL2_GRCh37_6:10990028-10990028_Missense-Mutation_SNP_C-C-T,ZNF7_GRCh37_8:146067177-146067178_Frame-Shift-Ins_INS_----A,SLC45A4_GRCh37_8:142228274-142228274_Frame-Shift-Del_DEL_C-C--,DNAH11_GRCh37_7:21631141-21631141_Frame-Shift-Del_DEL_A-A--,COL12A1_GRCh37_6:75798830-75798830_Missense-Mutation_SNP_C-C-G_C-C-T,KLHL31_GRCh37_6:53519759-53519759_Frame-Shift-Del_DEL_T-T--,MROH2B_GRCh37_5:41004528-41004528_Frame-Shift-Del_DEL_T-T--,RBM6_GRCh37_3:50099450-50099450_Frame-Shift-Del_DEL_A-A--,KIF15_GRCh37_3:44828026-44828026_Missense-Mutation_SNP_C-C-T,...,CEP63_GRCh37_3:134256014-134256014_Silent_SNP_G-G-A,MTUS2_GRCh37_13:29933476-29933476_Missense-Mutation_SNP_C-C-T,PRB2_GRCh37_12:11546745-11546745_Silent_SNP_A-A-G_A-A-C,SOHLH2_GRCh37_13:36776025-36776025_Missense-Mutation_SNP_T-T-C,FAM111A_GRCh37_11:58919526-58919526_Missense-Mutation_SNP_C-C-T,TP53_GRCh37_17:7577082-7577082_Missense-Mutation_SNP_C-C-T,DCSTAMP_GRCh37_8:105361304-105361304_Missense-Mutation_SNP_T-T-C,LCE1A_GRCh37_1:152800031-152800031_Missense-Mutation_SNP_A-A-C_A-A-G,TP53_GRCh37_17:7578403-7578403_Missense-Mutation_SNP_C-C-T_C-C-A,OLFM3_GRCh37_1:102302454-102302454_Missense-Mutation_SNP_A-A-C_A-A-G
8,999999,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,H(t),0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,...,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329
10,"H(s,t)",0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,...,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329
11,Pr,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.875000,1.000000,1.000000,1.000000
12,Pl,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000
13,"n(tR, NC)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
14,"n(tR, C)",8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,7.000000,8.000000,8.000000,8.000000
15,"n(tL, NC)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16,"n(tL, C)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
17,n(tR),8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,7.000000,8.000000,8.000000,8.000000


In [386]:
# # output right table with values
outputright = makegaintable(gainright)
outputright.iloc[outputright.shape[0]-11, 0] = 999999
outputright.iloc[outputright.shape[0]-11:,:].sort_values(by=outputright.shape[0]-11,axis=1,ascending=False)


/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:184: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 162
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:186: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 163
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:188: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 164
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42

,Patients,PGM5_GRCh37_9:70993145-70993145_Missense-Mutation_SNP_A-A-G,MVK_GRCh37_12:110019240-110019240_Frame-Shift-Del_DEL_C-C--,ZNF330_GRCh37_4:142143532-142143532_Frame-Shift-Del_DEL_A-A--,PLEKHA6_GRCh37_1:204228411-204228411_Frame-Shift-Del_DEL_C-C--,PIK3CA_GRCh37_3:178916876-178916876_Missense-Mutation_SNP_G-G-A,AXIN2_GRCh37_17:63532585-63532585_Frame-Shift-Del_DEL_C-C--,PTPRT_GRCh37_20:40739003-40739003_Frame-Shift-Del_DEL_G-G--,SI_GRCh37_3:164781250-164781250_Frame-Shift-Del_DEL_A-A--,LIPT1_GRCh37_2:99778781-99778781_Frame-Shift-Del_DEL_A-A--,...,KLHL31_GRCh37_6:53519759-53519759_Frame-Shift-Del_DEL_T-T--,MROH2B_GRCh37_5:41004528-41004528_Frame-Shift-Del_DEL_T-T--,KIF15_GRCh37_3:44828026-44828026_Missense-Mutation_SNP_C-C-T,FBXO48_GRCh37_2:68691370-68691370_Frame-Shift-Del_DEL_C-C--,SP140_GRCh37_2:231159025-231159025_Frame-Shift-Del_DEL_A-A--,SCN9A_GRCh37_2:167141099-167141099_Missense-Mutation_SNP_G-G-A,PATZ1_GRCh37_22:31737418-31737418_Intron_SNP_C-C-A,NTSR1_GRCh37_20:61386229-61386229_Missense-Mutation_SNP_C-C-T,PLAGL2_GRCh37_20:30789952-30789952_Frame-Shift-Del_DEL_G-G--,OR52A5_GRCh37_11:5153679-5153679_Frame-Shift-Del_DEL_A-A--
100,999999,0.064569,0.047964,0.047964,0.047964,0.047964,0.047964,0.047964,0.047964,0.047964,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
101,H(t),0.924819,0.924819,0.924819,0.924819,0.924819,0.924819,0.924819,0.924819,0.924819,...,0.924819,0.924819,0.924819,0.924819,0.924819,0.924819,0.924819,0.924819,0.924819,0.924819
102,"H(s,t)",0.860250,0.876854,0.876854,0.876854,0.876854,0.876854,0.876854,0.876854,0.876854,...,0.924819,0.924819,0.924819,0.924819,0.924819,0.924819,0.924819,0.924819,0.924819,0.924819
103,Pr,0.960000,0.970000,0.970000,0.970000,0.970000,0.970000,0.970000,0.970000,0.970000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
104,Pl,0.040000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
105,"n(tR, NC)",66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,...,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000
106,"n(tR, C)",30.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,...,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000
107,"n(tL, NC)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
108,"n(tL, C)",4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
109,n(tR),96.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [387]:
# reset the gain index's
outputog.iloc[108, 0] = 'Gain'
outputleft.iloc[outputleft.shape[0]-11, 0] = 'Gain'
outputright.iloc[outputright.shape[0]-11, 0] = 'Gain'

In [395]:
################################################## Groups 2 and 3 ##################################################
# get decision tree
gaindecisiontree2 = gaindecisiontree(hw8twothird, hw8threethird)

# classifier
hw8onethird = phiclassifer(gaindecisiontree2, hw8onethird)

# evaluate
hw8result1 = compare(hw8onethird)

# output results and the decision tree nodes
print("Nodes: ")
print(gaindecisiontree2)
hw8result1

/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True)


Nodes: 
['DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--', 'FARP1_GRCh37_13:99092237-99092237_Frame-Shift-Del_DEL_G-G--', 'UPF3A_GRCh37_13:115057211-115057211_Frame-Shift-Del_DEL_A-A--']


,Values
TP,1
FP,0
TN,31
FN,22
Accuracy,0.592593
Sensitivity,0.043478
Specificity,1.0
Missrate,0.956522
Precision,1.0
FDR,0.0


In [396]:
# output OG table with values
outputog = makegaintable(gainog)
outputog.iloc[108, 0] = 999999
outputog.iloc[108:,:].sort_values(by=outputog.shape[0]-11,axis=1,ascending=False)

/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:184: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 162
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:186: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 163
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:188: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 164
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42

,Patients,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--,RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--,PGM5_GRCh37_9:70993145-70993145_Missense-Mutation_SNP_A-A-G,LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del_DEL_T-T--,PHF2_GRCh37_9:96422612-96422612_Frame-Shift-Del_DEL_A-A--,UPF3A_GRCh37_13:115057211-115057211_Frame-Shift-Del_DEL_A-A--,KIAA0195_GRCh37_17:73491063-73491063_Frame-Shift-Del_DEL_C-C--,ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--,...,SCN9A_GRCh37_2:167055493-167055493_Missense-Mutation_SNP_G-G-A,PRB4_GRCh37_12:11461619-11461619_Nonsense-Mutation_SNP_C-C-A,SYTL1_GRCh37_1:27680355-27680356_3'UTR_DEL_CT-CT--,ALG10_GRCh37_12:34179403-34179404_Frame-Shift-Ins_INS_----T,MAP7D3_GRCh37_X:135314194-135314194_Frame-Shift-Del_DEL_G-G--,C1orf173_GRCh37_1:75072545-75072545_Missense-Mutation_SNP_G-G-A,PIK3CA_GRCh37_3:178936082-178936082_Missense-Mutation_SNP_G-G-A,MUC16_GRCh37_19:8993496-8993496_Missense-Mutation_SNP_G-G-A,CTNNB1_GRCh37_3:41266103-41266103_Missense-Mutation_SNP_G-G-A,GABBR1_GRCh37_6:29577144-29577144_Frame-Shift-Del_DEL_G-G--
108,999999,0.174536,0.136694,0.133490,0.118385,0.118385,0.100458,0.100458,0.100458,0.098720,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
109,H(t),0.887976,0.887976,0.887976,0.887976,0.887976,0.887976,0.887976,0.887976,0.887976,...,0.887976,0.887976,0.887976,0.887976,0.887976,0.887976,0.887976,0.887976,0.887976,0.887976
110,"H(s,t)",0.713440,0.751282,0.754486,0.769591,0.769591,0.787518,0.787518,0.787518,0.789256,...,0.887976,0.887976,0.887976,0.887976,0.887976,0.887976,0.887976,0.887976,0.887976,0.887976
111,Pr,0.907407,0.925926,0.898148,0.935185,0.935185,0.944444,0.944444,0.944444,0.916667,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
112,Pl,0.092593,0.074074,0.101852,0.064815,0.064815,0.055556,0.055556,0.055556,0.083333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
113,"n(tR, NC)",75.000000,75.000000,74.000000,75.000000,75.000000,75.000000,75.000000,75.000000,74.000000,...,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000
114,"n(tR, C)",23.000000,25.000000,23.000000,26.000000,26.000000,27.000000,27.000000,27.000000,25.000000,...,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000
115,"n(tL, NC)",0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
116,"n(tL, C)",10.000000,8.000000,10.000000,7.000000,7.000000,6.000000,6.000000,6.000000,8.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
117,n(tR),98.000000,100.000000,97.000000,101.000000,101.000000,102.000000,102.000000,102.000000,99.000000,...,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000


In [397]:
# output left table with values
outputleft = makegaintable(gainleft)
outputleft.iloc[outputleft.shape[0]-11, 0] = 999999
outputleft.iloc[outputleft.shape[0]-11:,:].sort_values(by=outputleft.shape[0]-11,axis=1,ascending=False)

/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:184: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 162
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:186: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 163
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:188: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 164
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42

,Patients,ELOVL2_GRCh37_6:10990028-10990028_Missense-Mutation_SNP_C-C-T,ZNF7_GRCh37_8:146067177-146067178_Frame-Shift-Ins_INS_----A,SLC45A4_GRCh37_8:142228274-142228274_Frame-Shift-Del_DEL_C-C--,DNAH11_GRCh37_7:21631141-21631141_Frame-Shift-Del_DEL_A-A--,COL12A1_GRCh37_6:75798830-75798830_Missense-Mutation_SNP_C-C-G_C-C-T,KLHL31_GRCh37_6:53519759-53519759_Frame-Shift-Del_DEL_T-T--,MROH2B_GRCh37_5:41004528-41004528_Frame-Shift-Del_DEL_T-T--,RBM6_GRCh37_3:50099450-50099450_Frame-Shift-Del_DEL_A-A--,KIF15_GRCh37_3:44828026-44828026_Missense-Mutation_SNP_C-C-T,...,CEP63_GRCh37_3:134256014-134256014_Silent_SNP_G-G-A,MTUS2_GRCh37_13:29933476-29933476_Missense-Mutation_SNP_C-C-T,PRB2_GRCh37_12:11546745-11546745_Silent_SNP_A-A-G_A-A-C,SOHLH2_GRCh37_13:36776025-36776025_Missense-Mutation_SNP_T-T-C,FAM111A_GRCh37_11:58919526-58919526_Missense-Mutation_SNP_C-C-T,TP53_GRCh37_17:7577082-7577082_Missense-Mutation_SNP_C-C-T,DCSTAMP_GRCh37_8:105361304-105361304_Missense-Mutation_SNP_T-T-C,LCE1A_GRCh37_1:152800031-152800031_Missense-Mutation_SNP_A-A-C_A-A-G,TP53_GRCh37_17:7578403-7578403_Missense-Mutation_SNP_C-C-T_C-C-A,OLFM3_GRCh37_1:102302454-102302454_Missense-Mutation_SNP_A-A-C_A-A-G
10,999999,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
11,H(t),0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,...,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329
12,"H(s,t)",0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,...,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329
13,Pr,1.000000,0.900000,0.900000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.900000,1.000000,1.000000,1.000000
14,Pl,0.000000,0.100000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000
15,"n(tR, NC)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16,"n(tR, C)",10.000000,9.000000,9.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,9.000000,10.000000,10.000000,10.000000
17,"n(tL, NC)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
18,"n(tL, C)",0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
19,n(tR),10.000000,9.000000,9.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,9.000000,10.000000,10.000000,10.000000


In [398]:
# output right table with values
outputright = makegaintable(gainright)
outputright.iloc[outputright.shape[0]-11, 0] = 999999
outputright.iloc[outputright.shape[0]-11:,:].sort_values(by=outputright.shape[0]-11,axis=1,ascending=False)


/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:184: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 162
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:186: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 163
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:188: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 164
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42

,Patients,UPF3A_GRCh37_13:115057211-115057211_Frame-Shift-Del_DEL_A-A--,ZC3H18_GRCh37_16:88691141-88691141_Frame-Shift-Del_DEL_C-C--,FCRL3_GRCh37_1:157666960-157666960_Frame-Shift-Del_DEL_T-T--,BMPR2_GRCh37_2:203420130-203420130_Frame-Shift-Del_DEL_A-A--,GYLTL1B_GRCh37_11:45948373-45948373_Frame-Shift-Del_DEL_C-C--,GLI3_GRCh37_7:42005573-42005573_Frame-Shift-Del_DEL_G-G--,ZNF330_GRCh37_4:142143532-142143532_Frame-Shift-Del_DEL_A-A--,PGM5_GRCh37_9:70993145-70993145_Missense-Mutation_SNP_A-A-G,TBL1XR1_GRCh37_3:176769342-176769342_Frame-Shift-Del_DEL_T-T--,...,PCDHB6_GRCh37_5:140530600-140530600_Frame-Shift-Del_DEL_C-C--,FAM193A_GRCh37_4:2698177-2698177_Frame-Shift-Del_DEL_A-A--,ANK2_GRCh37_4:114274959-114274959_Frame-Shift-Del_DEL_A-A--,ATP6V1B1_GRCh37_2:71191573-71191573_Frame-Shift-Del_DEL_C-C--,EIF5B_GRCh37_2:99976805-99976805_Frame-Shift-Del_DEL_A-A--,RHOQ_GRCh37_2:46808174-46808174_Frame-Shift-Del_DEL_A-A--,C2orf53_GRCh37_2:27361028-27361028_Missense-Mutation_SNP_C-C-T,DGKD_GRCh37_2:234365952-234365952_Frame-Shift-Del_DEL_G-G--,RIF1_GRCh37_2:152326345-152326345_Frame-Shift-Del_DEL_A-A--,SESTD1_GRCh37_2:179982318-179982318_Nonsense-Mutation_SNP_G-G-A
98,999999,0.089526,0.089526,0.066305,0.066305,0.066305,0.066305,0.066305,0.066305,0.066305,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
99,H(t),0.786104,0.786104,0.786104,0.786104,0.786104,0.786104,0.786104,0.786104,0.786104,...,0.786104,0.786104,0.786104,0.786104,0.786104,0.786104,0.786104,0.786104,0.786104,0.786104
100,"H(s,t)",0.696578,0.696578,0.719799,0.719799,0.719799,0.719799,0.719799,0.719799,0.719799,...,0.786104,0.786104,0.786104,0.786104,0.786104,0.786104,0.786104,0.786104,0.786104,0.786104
101,Pr,0.959184,0.959184,0.969388,0.969388,0.969388,0.969388,0.969388,0.969388,0.969388,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
102,Pl,0.040816,0.040816,0.030612,0.030612,0.030612,0.030612,0.030612,0.030612,0.030612,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
103,"n(tR, NC)",75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,...,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000
104,"n(tR, C)",19.000000,19.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,...,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000
105,"n(tL, NC)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
106,"n(tL, C)",4.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
107,n(tR),94.000000,94.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,...,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000


In [399]:
# reset the gain index's
outputog.iloc[108, 0] = 'Gain'
outputleft.iloc[outputleft.shape[0]-11, 0] = 'Gain'
outputright.iloc[outputright.shape[0]-11, 0] = 'Gain'

In [400]:

################################################## Groups 1 and 2 ##################################################
# get decision tree
gaindecisiontree3 = gaindecisiontree(hw8onethird, hw8twothird)

# classifier
hw8threethird = phiclassifer(gaindecisiontree3, hw8threethird)

# evaluate
hw8result3 = compare(hw8threethird)
# output results and the decision tree nodes
print("Nodes: ")
print(gaindecisiontree3)
hw8result3

/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True)


Nodes: 
['ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--', 'FARP1_GRCh37_13:99092237-99092237_Frame-Shift-Del_DEL_G-G--', 'UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--']


,Values
TP,2
FP,1
TN,34
FN,17
Accuracy,0.666667
Sensitivity,0.105263
Specificity,0.971429
Missrate,0.894737
Precision,0.666667
FDR,0.333333


In [401]:
# output OG table with values
outputog = makegaintable(gainog)
outputog.iloc[108, 0] = 999999
outputog.iloc[108:,:].sort_values(by=outputog.shape[0]-11,axis=1,ascending=False)

/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:184: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 162
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:186: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 163
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:188: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 164
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42

,Patients,ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A--,UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--,RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--,TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_C-C-T,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--,AKAP7_GRCh37_6:131481276-131481276_Frame-Shift-Del_DEL_A-A--,UPF3A_GRCh37_13:115057211-115057211_Frame-Shift-Del_DEL_A-A--,KDM1B_GRCh37_6:18222307-18222307_3'UTR_DEL_A-A--,LHX1_GRCh37_17:35300395-35300395_Frame-Shift-Del_DEL_C-C--,...,HAUS3_GRCh37_4:2242749-2242749_5'UTR_DEL_A-A--,NOL9_GRCh37_1:6585790-6585790_3'UTR_DEL_A-A--,CSTF1_GRCh37_20:54978601-54978601_Missense-Mutation_SNP_G-G-A,EPHA3_GRCh37_3:89499363-89499363_Frame-Shift-Del_DEL_C-C--,ZNF445_GRCh37_3:44492863-44492863_Frame-Shift-Del_DEL_C-C--,PTPRJ_GRCh37_11:48146561-48146561_Missense-Mutation_SNP_G-G-A,PDCD6IP_GRCh37_3:33866819-33866819_Missense-Mutation_SNP_A-A-T,KCNC2_GRCh37_12:75444468-75444468_Silent_SNP_A-A-G_A-A-C,FAM86HP_GRCh37_3:129823065-129823065_RNA_SNP_G-G-A,ZC3H4_GRCh37_19:47572412-47572412_Missense-Mutation_SNP_C-C-G
108,999999,0.139498,0.122808,0.100651,0.090396,0.090396,0.074647,0.074647,0.074647,0.074647,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
109,H(t),0.927278,0.927278,0.927278,0.927278,0.927278,0.927278,0.927278,0.927278,0.927278,...,0.927278,0.927278,0.927278,0.927278,0.927278,0.927278,0.927278,0.927278,0.927278,0.927278
110,"H(s,t)",0.787780,0.804470,0.826628,0.836882,0.836882,0.852631,0.852631,0.852631,0.852631,...,0.927278,0.927278,0.927278,0.927278,0.927278,0.927278,0.927278,0.927278,0.927278,0.927278
111,Pr,0.916667,0.925926,0.907407,0.944444,0.944444,0.953704,0.953704,0.953704,0.953704,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
112,Pl,0.083333,0.074074,0.092593,0.055556,0.055556,0.046296,0.046296,0.046296,0.046296,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
113,"n(tR, NC)",71.000000,71.000000,70.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,...,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000
114,"n(tR, C)",28.000000,29.000000,28.000000,31.000000,31.000000,32.000000,32.000000,32.000000,32.000000,...,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000
115,"n(tL, NC)",0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
116,"n(tL, C)",9.000000,8.000000,9.000000,6.000000,6.000000,5.000000,5.000000,5.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
117,n(tR),99.000000,100.000000,98.000000,102.000000,102.000000,103.000000,103.000000,103.000000,103.000000,...,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000


In [402]:
# output left table with values
outputleft = makegaintable(gainleft)
outputleft.iloc[outputleft.shape[0]-11, 0] = 999999
outputleft.iloc[outputleft.shape[0]-11:,:].sort_values(by=outputleft.shape[0]-11,axis=1,ascending=False)

/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:184: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 162
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:186: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 163
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:188: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 164
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42

,Patients,DMRT1_GRCh37_9:841907-841907_Silent_SNP_G-G-A,NWD1_GRCh37_19:16860457-16860457_Frame-Shift-Del_DEL_C-C--,FCRLB_GRCh37_1:161697204-161697204_Missense-Mutation_SNP_G-G-A,GNAS_GRCh37_20:57485424-57485424_Missense-Mutation_SNP_C-C-T,TRIB2_GRCh37_2:12858399-12858399_5'UTR_DEL_T-T--,KANSL1L_GRCh37_2:211019062-211019062_Frame-Shift-Del_DEL_A-A--,SERPINE2_GRCh37_2:224866442-224866442_Missense-Mutation_SNP_G-G-A,TBL1XR1_GRCh37_3:176769342-176769342_Frame-Shift-Del_DEL_T-T--,IL7R_GRCh37_5:35867541-35867541_Frame-Shift-Del_DEL_A-A--,...,SPOCK1_GRCh37_5:136324212-136324212_Missense-Mutation_SNP_T-T-A_T-T-G,CSMD1_GRCh37_8:2807839-2807839_Missense-Mutation_SNP_A-A-C,IFNA17_GRCh37_9:21227998-21227998_Missense-Mutation_SNP_A-A-T_A-A-G,ITPR1_GRCh37_3:4699844-4699844_Missense-Mutation_SNP_C-C-T,PCDHA4_GRCh37_5:140188676-140188676_Missense-Mutation_SNP_G-G-A,RTN2_GRCh37_19:45996513-45996513_Frame-Shift-Del_DEL_G-G--,ERV3-1_GRCh37_7:64452191-64452191_Missense-Mutation_SNP_A-A-G_A-A-T,FLG_GRCh37_1:152285533-152285533_Missense-Mutation_SNP_C-C-T,CHD8_GRCh37_14:21862288-21862288_Missense-Mutation_SNP_C-C-T,OLFM3_GRCh37_1:102302454-102302454_Missense-Mutation_SNP_A-A-C_A-A-G
9,999999,2.168404e-19,2.168404e-19,2.168404e-19,2.168404e-19,2.168404e-19,2.168404e-19,2.168404e-19,2.168404e-19,2.168404e-19,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10,H(t),1.328771e-03,1.328771e-03,1.328771e-03,1.328771e-03,1.328771e-03,1.328771e-03,1.328771e-03,1.328771e-03,1.328771e-03,...,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329
11,"H(s,t)",1.328771e-03,1.328771e-03,1.328771e-03,1.328771e-03,1.328771e-03,1.328771e-03,1.328771e-03,1.328771e-03,1.328771e-03,...,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329,0.001329
12,Pr,8.888889e-01,8.888889e-01,8.888889e-01,8.888889e-01,8.888889e-01,8.888889e-01,8.888889e-01,8.888889e-01,8.888889e-01,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.777778,1.000000,1.000000,1.000000,1.000000
13,Pl,1.111111e-01,1.111111e-01,1.111111e-01,1.111111e-01,1.111111e-01,1.111111e-01,1.111111e-01,1.111111e-01,1.111111e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.222222,0.000000,0.000000,0.000000,0.000000
14,"n(tR, NC)",0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15,"n(tR, C)",8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,...,9.000000,9.000000,9.000000,9.000000,9.000000,7.000000,9.000000,9.000000,9.000000,9.000000
16,"n(tL, NC)",0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
17,"n(tL, C)",1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
18,n(tR),8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,...,9.000000,9.000000,9.000000,9.000000,9.000000,7.000000,9.000000,9.000000,9.000000,9.000000


In [403]:
# output right table with values
outputright = makegaintable(gainright)
outputright.iloc[outputright.shape[0]-11, 0] = 999999
outputright.iloc[outputright.shape[0]-11:,:].sort_values(by=outputright.shape[0]-11,axis=1,ascending=False)


/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:184: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 162
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:186: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 163
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42866/3758556516.py:188: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gaindf = gaindf.append(pd.Series(), ignore_index=True) # 164
/var/folders/n8/6m_nbq0n6rx2201gyz79k4f40000gn/T/ipykernel_42

,Patients,UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--,PGM5_GRCh37_9:70993145-70993145_Missense-Mutation_SNP_A-A-G,LHX1_GRCh37_17:35300395-35300395_Frame-Shift-Del_DEL_C-C--,KLF3_GRCh37_4:38690460-38690460_Frame-Shift-Del_DEL_A-A--,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--,CADM1_GRCh37_11:115047097-115047097_3'UTR_DEL_A-A--,MUC16_GRCh37_19:8993496-8993496_Missense-Mutation_SNP_G-G-A,SYTL1_GRCh37_1:27680355-27680356_3'UTR_DEL_CT-CT--,BEST3_GRCh37_12:70091534-70091534_Frame-Shift-Del_DEL_A-A--,...,PRDM2_GRCh37_1:14108436-14108436_Silent_SNP_C-C-T,INTS3_GRCh37_1:153723713-153723713_Missense-Mutation_SNP_C-C-T,LIPT1_GRCh37_2:99778781-99778781_Frame-Shift-Del_DEL_A-A--,TMEM79_GRCh37_1:156255497-156255498_Frame-Shift-Ins_INS_----C,ISG20L2_GRCh37_1:156694023-156694023_Frame-Shift-Del_DEL_G-G--,KCNJ10_GRCh37_1:160012018-160012018_Frame-Shift-Del_DEL_G-G--,NLRC5_GRCh37_16:57113198-57113201_Frame-Shift-Del_DEL_AGAT-AGAT--,RAB3GAP2_GRCh37_1:220324483-220324483_3'UTR_DEL_A-A--,PPCS_GRCh37_1:42925658-42925658_3'UTR_DEL_A-A--,RBM24_GRCh37_6:17292140-17292140_Silent_SNP_C-C-T
99,999999,0.076746,0.076746,0.056933,0.056933,0.056933,0.056933,0.037551,0.037551,0.037551,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
100,H(t),0.859276,0.859276,0.859276,0.859276,0.859276,0.859276,0.859276,0.859276,0.859276,...,0.859276,0.859276,0.859276,0.859276,0.859276,0.859276,0.859276,0.859276,0.859276,0.859276
101,"H(s,t)",0.782530,0.782530,0.802343,0.802343,0.802343,0.802343,0.821724,0.821724,0.821724,...,0.859276,0.859276,0.859276,0.859276,0.859276,0.859276,0.859276,0.859276,0.859276,0.859276
102,Pr,0.959596,0.959596,0.969697,0.969697,0.969697,0.969697,0.979798,0.979798,0.979798,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
103,Pl,0.040404,0.040404,0.030303,0.030303,0.030303,0.030303,0.020202,0.020202,0.020202,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
104,"n(tR, NC)",71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,...,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000
105,"n(tR, C)",24.000000,24.000000,25.000000,25.000000,25.000000,25.000000,26.000000,26.000000,26.000000,...,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000
106,"n(tL, NC)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
107,"n(tL, C)",4.000000,4.000000,3.000000,3.000000,3.000000,3.000000,2.000000,2.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
108,n(tR),95.000000,95.000000,96.000000,96.000000,96.000000,96.000000,97.000000,97.000000,97.000000,...,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000


In [404]:
# reset the gain index's
outputog.iloc[108, 0] = 'Gain'
outputleft.iloc[outputleft.shape[0]-11, 0] = 'Gain'
outputright.iloc[outputright.shape[0]-11, 0] = 'Gain'

In [405]:
# avg of the results
hw8pt2avg = pd.DataFrame(columns=['Values'])
namelist = ['TP','FP','TN','FN','accuracy','sensitivity','specificity','missrate','precision','FDR','FOR']
for i in range(0, hw6result1.shape[0]):
    x = (hw8result1.iat[i, 0] + hw8result2.iat[i, 0] + hw8result3.iat[i, 0]) / 3
    hw8pt2avg.loc[namelist[i]] = x

hw8pt2avg

,Values
TP,1.666667
FP,0.333333
TN,35.000000
FN,17.000000
accuracy,0.679012
sensitivity,0.097200
specificity,0.990476
missrate,0.902800
precision,0.888889
FDR,0.111111


In [1339]:
##################################################################### HW 9 ###############################################################################
# copy of the df for this assignment
hw9df = df.iloc[0:162, 0:1411]
outofbag = []
outofbagsize = 0


In [1340]:
# delete mutations with <= 4 TP-FP
hw9df = setconfusion(hw9df)

# get list of columns to delete
dellater = []
for i in range(1, hw9df.shape[1]):
    if hw9df.iloc[166, i] <= 4:
        dellater.append(hw9df.columns[i])

# delete columns
hw9df = hw9df.drop(dellater, axis=1)

#delete useless rows
hw9df = hw9df.drop(labels=[162, 163, 164, 165, 166], axis=0)
hw9df


,Patients,OR5M3_GRCh37_11:56237709-56237709_Frame-Shift-Del_DEL_T-T--,MVK_GRCh37_12:110019240-110019240_Frame-Shift-Del_DEL_C-C--,TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_C-C-T,KLHL14_GRCh37_18:30349862-30349862_Frame-Shift-Del_DEL_G-G--,NFASC_GRCh37_1:204924033-204924033_Frame-Shift-Del_DEL_C-C--,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--,C7orf50_GRCh37_7:1037311-1037311_Frame-Shift-Del_DEL_G-G--,ZC3H18_GRCh37_16:88691141-88691141_Frame-Shift-Del_DEL_C-C--,...,UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T--,MIR1303_GRCh37_5:154065380-154065383_RNA_DEL_TTTA-TTTA--,AKAP7_GRCh37_6:131481276-131481276_Frame-Shift-Del_DEL_A-A--,UPF3A_GRCh37_13:115057211-115057211_Frame-Shift-Del_DEL_A-A--,SVIL_GRCh37_10:29760116-29760116_Frame-Shift-Del_DEL_C-C--,C11orf35_GRCh37_11:558069-558069_RNA_DEL_G-G--,RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--,LHX1_GRCh37_17:35300395-35300395_Frame-Shift-Del_DEL_C-C--,KIAA0195_GRCh37_17:73491063-73491063_Frame-Shift-Del_DEL_C-C--,MXRA8_GRCh37_1:1290110-1290110_Frame-Shift-Del_DEL_G-G--
FARP1_GRCh37_13:99092237-99092237_Frame-Shift-Del_DEL_G-G--,,,,,,,,,,,,,,,,,,,,,
0,C1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NC1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,C3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NC2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,C55,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
158,NC104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159,NC105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1288]:
# given a dataframe with tp-fp we create a decision tree
# for each node we take 37 mutations from the df
def randtree(hw9df):
    # overall list (root, left, right)
    listree = []
    
    # get the random patients
    decdf = hw9df.iloc[0:162, 0:1411].sample(n=162,replace=True)
    
    # reset index 
    decdf = decdf.reset_index(drop=True)
    
    # save the patients
    patients = decdf.iloc[0:162, 0]

    # get randome 15 mutations
    decdf = decdf.iloc[0:162, 1:hw9df.shape[1]].sample(n=15, replace=False, axis=1)

    # add patients back
    decdf.insert(0, 'Patients', patients)

    # get the tp-fp
    decdf = setconfusion(decdf)

    # delete the rows we dont need
    decdf = decdf.drop(labels=[decdf.shape[0] - 5, decdf.shape[0] - 4, decdf.shape[0] - 3, decdf.shape[0] - 2], axis=0)

    # reset the index
    decdf = decdf.reset_index(drop=True)
    
    # for avg/out of bag
    # get patients not in the OG set
    something = hw9df[~hw9df['Patients'].isin(patients)]
    # reset index so shape[] isnt scuffed
    something = something.reset_index(drop=True)
    # list of out of bag datasets (list of df's)
    global outofbag
    outofbag.append(something)
    
    # output the size
    # print("this is out of bag size", something.shape[0])
    global outofbagsize
    # for the avg at the end
    outofbagsize+= something.shape[0]
    
    # get the top mutation
    top = decdf.iloc[decdf.shape[0] - 1, 1:decdf.shape[1]].sort_values(ascending=False).index
    root = top[0]
    
    # get the root in the tree
    listree.append(root)
    
    # split the df by the root (list of patients)
    nodeA = [] 
    nodeB = []
    counter = 0
    for i in range(0, decdf.shape[0] - 1):
        if decdf.at[i, root] == 1: # it has the root
            nodeA.append(decdf.iat[i,0]) # gets the patient
            counter += 1
        if decdf.at[i, root] == 0: # it doesnt have the root
            nodeB.append(decdf.iat[i,0]) # gets the patient
            counter += 1
    # end of i

    # get dataframes for the split
    decdfA = hw9df[~hw9df['Patients'].isin(nodeB)]
    decdfB = hw9df[~hw9df['Patients'].isin(nodeA)]

    # get random 37 mutations
    decdfA = decdfA.iloc[0:len(nodeA), 1:hw9df.shape[1]].sample(n=15,replace=False,axis=1)
    decdfB = decdfB.iloc[0:len(nodeB), 1:hw9df.shape[1]].sample(n=15,replace=False,axis=1)

    # reset index
    decdfA = decdfA.reset_index(drop=True)
    decdfB = decdfB.reset_index(drop=True)

    # add patients back
    decdfA.insert(0, 'Patients', nodeA)
    decdfB.insert(0, 'Patients', nodeB)

    # get TP-FP
    decdfA = setconfusion(decdfA)
    decdfB = setconfusion(decdfB)
    
    # delete useless rows
    decdfA = decdfA.drop(labels=[decdfA.shape[0] - 5, decdfA.shape[0] - 4, decdfA.shape[0] - 3, decdfA.shape[0] - 2], axis=0)
    decdfB = decdfB.drop(labels=[decdfB.shape[0] - 5, decdfB.shape[0] - 4, decdfB.shape[0] - 3, decdfB.shape[0] - 2], axis=0)

    # get the node
    topA = decdfA.iloc[decdfA.shape[0] - 1, 1:decdfA.shape[1]].sort_values(ascending=False).index
    listree.append(topA[0])
    topB = decdfB.iloc[decdfB.shape[0] - 1, 1:decdfB.shape[1]].sort_values(ascending=False).index
    listree.append(topB[0])
    
    # return the list [root, left, right]
    return listree
# end of function

In [1190]:
# get tree
decisionlist = randtree(hw9df)
# print tree
print(decisionlist)
# get out of bag df
outofbag[0] = outofbag[0].reset_index(drop=True)
outofbag[0]

this is out of bag size 54
['OR5M3_GRCh37_11:56237709-56237709_Frame-Shift-Del_DEL_T-T--', 'CCAR2_GRCh37_8:22472975-22472975_Frame-Shift-Del_DEL_C-C--', 'OSMR_GRCh37_5:38923329-38923329_Frame-Shift-Del_DEL_A-A--']


,Patients,FARP1_GRCh37_13:99092237-99092237_Frame-Shift-Del_DEL_G-G--,QKI_GRCh37_6:163987695-163987695_Intron_DEL_T-T--,NHLRC2_GRCh37_10:115662308-115662308_Frame-Shift-Del_DEL_A-A--,PDZD8_GRCh37_10:119042898-119042898_Frame-Shift-Del_DEL_T-T--,EBF3_GRCh37_10:131666125-131666125_Frame-Shift-Del_DEL_C-C--,MASTL_GRCh37_10:27459272-27459272_Frame-Shift-Del_DEL_A-A--,PTCHD3_GRCh37_10:27687763-27687763_Frame-Shift-Del_DEL_A-A--,PFKP_GRCh37_10:3151612-3151612_Silent_SNP_C-C-T,CREM_GRCh37_10:35456559-35456559_Intron_DEL_T-T--,...,CDH23_GRCh37_10:73405660-73405660_Missense-Mutation_SNP_G-G-A,SLC26A11_GRCh37_17:78195370-78195370_Missense-Mutation_SNP_C-C-T,MME_GRCh37_3:154866382-154866382_Missense-Mutation_SNP_A-A-T_A-A-C,CADPS_GRCh37_3:62459960-62459960_Missense-Mutation_SNP_G-G-T,SCN4A_GRCh37_17:62049998-62049998_Silent_SNP_G-G-A,FPR2_GRCh37_19:52272624-52272624_Missense-Mutation_SNP_G-G-A,OLFM3_GRCh37_1:102302454-102302454_Missense-Mutation_SNP_A-A-C_A-A-G,C,NC,Class
0,NC1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
1,NC2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
2,NC6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
3,NC10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
4,NC11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
5,NC12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
6,C5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,23,NC
7,NC15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
8,NC19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
9,NC22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC


In [1261]:
# remove the stuff we just did in the previous cell
# outofbag.pop()
# outofbagsize = 0

# amount is how many trees we are going to produce
def randomforest(amount, hw9df):
    # List of list with trees
    treelist = []
    for i in range(0, amount):
        tmp = randtree(hw9df)
        treelist.append(tmp)
    # end of i
    
    # return the list of list (list of trees)
    return treelist
# end of function

In [1341]:
# get the list of list, aka make the random forest
randforest = randomforest(25, hw9df)

# output
print("This is the average of the out of bag size:", (outofbagsize / 25))
randforest

This is the average of the out of bag size: 60.6


[['RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--',
  'UPF3A_GRCh37_13:115057211-115057211_Frame-Shift-Del_DEL_A-A--',
  'RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--'],
 ["KDM1B_GRCh37_6:18222307-18222307_3'UTR_DEL_A-A--",
  'ZNF330_GRCh37_4:142143532-142143532_Frame-Shift-Del_DEL_A-A--',
  'NFASC_GRCh37_1:204924033-204924033_Frame-Shift-Del_DEL_C-C--'],
 ['DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--',
  "KDM1B_GRCh37_6:18222307-18222307_3'UTR_DEL_A-A--",
  'KIAA0195_GRCh37_17:73491063-73491063_Frame-Shift-Del_DEL_C-C--'],
 ['LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del_DEL_T-T--',
  'PGM5_GRCh37_9:70993145-70993145_Missense-Mutation_SNP_A-A-G',
  'TRIM51_GRCh37_11:55653610-55653610_Frame-Shift-Del_DEL_A-A--'],
 ['ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--',
  'ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--',
  'PHF2_GRCh37_9:96422612-96422612_Frame-Shift-Del_DEL_A-A--'],
 ['MIR1303_GRCh37_5:154065380-154065383_R

In [1320]:
# make X amount of random forests then go through and see how many times
# a node appears within the random forest
def getcommon(randforest):
# go through the list and see how many times the root or nodes appear 
    dictcommon = {}
    for j in range(0, len(randforest)):
        for i in range(1, len(randforest)):
            if randforest[j][0] == randforest[i][0]: # we got 1 match and we add that
                if randforest[j][0] not in dictcommon.keys():
                    dictcommon[randforest[j][0]] = 1
                else: # its already in 
                    dictcommon[randforest[j][0]] += 1
                for k in range(i + 1, len(randforest)): # check for more
                    if randforest[j][0] == randforest[k][0] and randforest[k][0] in dictcommon.keys():
                            dictcommon[randforest[j][0]] += 1
                    else:
                        j = k
                        i = j + 1
    # end of root check
    for j in range(0, len(randforest)):
        for i in range(1, len(randforest)):
            if randforest[j][1] == randforest[i][1]:
                if randforest[j][1] not in dictcommon.keys():
                    dictcommon[randforest[j][1]] = 1
                else: # its already in 
                    dictcommon[randforest[j][1]] += 1
                for k in range(i + 1, len(randforest)):
                    if randforest[j][1] == randforest[k][1] and randforest[k][1] in dictcommon.keys():
                        dictcommon[randforest[j][1]] += 1
                    else:
                        j = k
                        i = j + 1
    # end of left check
    for j in range(0, len(randforest)):
        for i in range(1, len(randforest)):
            if randforest[j][2] == randforest[i][2]:
                if randforest[j][2] not in dictcommon.keys():
                    dictcommon[randforest[j][2]] = 1
                else: # its already in 
                    dictcommon[randforest[j][2]] += 1
                for k in range(i + 1, len(randforest)):
                    if randforest[j][2] == randforest[k][2] and randforest[k][2] in dictcommon.keys():
                        dictcommon[randforest[j][2]] += 1
                    else:
                        j = k
                        i = j + 1
    # end of checks
    return dictcommon
# end of function

def manycommon(amount):
    commondict = {}
    for i in range(amount):
        tmpdict = {}
        # get a buncha random forest and find the best mutation
        randforest = randomforest(25, hw9df)
        # send the forest to getcommon
        tmpdict = getcommon(randforest)
        # see if that is already in the map if so increment
        for k, v in tmpdict.items():
            if k in commondict.keys():
                print(v)
                commondict[k] += v
            else: # its not in the dict
                print(v)
                commondict[k] = v
    
    # print out the dict
    for k, v in commondict.items():
        print(k,v)
# end of func

manycommon(1)

# Picked mutations are:
# DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--
# ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--
# RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C--
# UPF3A_GRCh37_13:115057211-115057211_Frame-Shift-Del_DEL_A-A--

638
311
6
1
2
3
53
108
18
19
12
8
4
2
1
DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C-- 638
ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G-- 311
UBR5_GRCh37_8:103289349-103289349_Frame-Shift-Del_DEL_T-T-- 6
ACVR2A_GRCh37_2:148683686-148683686_Frame-Shift-Del_DEL_A-A-- 1
PGM5_GRCh37_9:70993145-70993145_Missense-Mutation_SNP_A-A-G 2
PHF2_GRCh37_9:96422612-96422612_Frame-Shift-Del_DEL_A-A-- 3
UPF3A_GRCh37_13:115057211-115057211_Frame-Shift-Del_DEL_A-A-- 53
RNF43_GRCh37_17:56435161-56435161_Frame-Shift-Del_DEL_C-C-- 108
TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_C-C-T 18
KDM1B_GRCh37_6:18222307-18222307_3'UTR_DEL_A-A-- 19
ZNF330_GRCh37_4:142143532-142143532_Frame-Shift-Del_DEL_A-A-- 12
LHX1_GRCh37_17:35300395-35300395_Frame-Shift-Del_DEL_C-C-- 8
LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del_DEL_T-T-- 4
KIAA0195_GRCh37_17:73491063-73491063_Frame-Shift-Del_DEL_C-C-- 2
RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T-- 1


In [1194]:
# given a input classify it
# get user input
patient_input = input("Give us a patient: ")
input_row = df.loc[df['Patients'] == patient_input]
# classification = randforestclassifer(hw9df, patient_input, randforest)
while(patient_input != 'quit'):
    # go through the rand forest
    Camount, NCamount = 0, 0
    for i in range(0, len(randforest)):
        if(input_row[randforest[i][0]].values == 1):
            # got the root
            if(input_row[randforest[i][1]].values == 1):
                Camount += 1
            else: # doesnt have it
                NCamount += 1
        else: # dont have root
            if(input_row[randforest[i][2]].values == 1):
                Camount += 1
            else: # doesnt have it
                NCamount += 1
    # end of for
        
    # check amounts for output
    if(NCamount > Camount):
        print("We labeled:",patient_input,"as NC")
    else:
        print("We labeled:",patient_input,"as C")
    
    # get more patients if needed    
    del patient_input
    del input_row
    patient_input = input("Give the name of the Patient: ")
    input_row = df.loc[df['Patients'] == patient_input]
# end of while

In [1195]:
# give the out of bag data set classify them
def randomeval(tmpdf):
    # make 2 columns for c amount vs nc amount
    # go through each patient and eval against randforest
    # for k in range(0, len(evaldf)):
        # tmpdf = evaldf[k]
    
    tmpdf['C'] = 0
    tmpdf['NC'] = 0
    columns = tmpdf.columns
    c = columns.get_loc('C')
    nc = columns.get_loc('NC')
    for i in range(0, tmpdf.shape[0]):
        for j in range(0, len(randforest)):
            x = columns.get_loc(randforest[j][0])
            if(tmpdf.iat[i, x] == 1): # has root
                x = columns.get_loc(randforest[j][1])
                if(tmpdf.iat[i, x] == 1): # has left, C
                    tmpdf.iat[i, c] += 1
                else: # doesnt have left, NC
                    tmpdf.iat[i, nc] += 1
            else: # doesnt have root
                x = columns.get_loc(randforest[j][2])
                if(tmpdf.iat[i, x] == 1): # has right, C
                    tmpdf.iat[i, c] += 1
                else: # doesnt have right, NC
                    tmpdf.iat[i, nc] += 1
            # end of else
        # end of j
    # end of i
        # evaldf[k] = tmpdf
    # end of k
    
    # classify
    tmpdf['Class'] = ''
    columns = tmpdf.columns
    classidx = columns.get_loc('Class')
    for i in range(0, tmpdf.shape[0]):
        if tmpdf.iat[i, c] > tmpdf.iat[i, nc]: # C > NC
            tmpdf.iat[i, classidx] = 'C'
        else: 
            tmpdf.iat[i, classidx] = 'NC'
    return tmpdf
# end of function

In [1342]:
# evaluate
tmp = outofbag
for i in range(0, len(tmp)):
    tmp[i] = randomeval(tmp[i])
    
tmp[0]

,Patients,OR5M3_GRCh37_11:56237709-56237709_Frame-Shift-Del_DEL_T-T--,MVK_GRCh37_12:110019240-110019240_Frame-Shift-Del_DEL_C-C--,TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_C-C-T,KLHL14_GRCh37_18:30349862-30349862_Frame-Shift-Del_DEL_G-G--,NFASC_GRCh37_1:204924033-204924033_Frame-Shift-Del_DEL_C-C--,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shift-Del_DEL_G-G--,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-Del_DEL_C-C--,C7orf50_GRCh37_7:1037311-1037311_Frame-Shift-Del_DEL_G-G--,ZC3H18_GRCh37_16:88691141-88691141_Frame-Shift-Del_DEL_C-C--,...,UPF3A_GRCh37_13:115057211-115057211_Frame-Shift-Del_DEL_A-A--,SVIL_GRCh37_10:29760116-29760116_Frame-Shift-Del_DEL_C-C--,C11orf35_GRCh37_11:558069-558069_RNA_DEL_G-G--,RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--,LHX1_GRCh37_17:35300395-35300395_Frame-Shift-Del_DEL_C-C--,KIAA0195_GRCh37_17:73491063-73491063_Frame-Shift-Del_DEL_C-C--,MXRA8_GRCh37_1:1290110-1290110_Frame-Shift-Del_DEL_G-G--,C,NC,Class
0,C3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
1,NC2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
2,NC4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
3,NC6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
4,NC7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
5,NC10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
6,NC13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
7,C6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,25,NC
8,C8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,21,NC
9,C10,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2,23,NC


In [1343]:
# get the tp, fp, FDR, etc
for i in range(0, len(tmp)):
    tmp[i] = compare(tmp[i])
    

C3
NC2
NC4
NC6
NC7
NC10
NC13
C6
C8
C10
NC24
NC25
C14
NC28
NC35
NC36
NC37
C20
NC39
C21
NC41
NC45
NC46
NC52
NC53
NC55
NC57
NC61
NC62
C32
NC68
NC71
NC72
NC73
NC84
NC85
NC86
NC89
NC91
NC92
C41
C42
NC94
C43
C46
C47
NC100
C48
C52
NC102
C53
C55
C56
C1
NC6
NC7
NC8
C4
NC11
NC13
C5
C6
NC15
NC17
NC19
NC20
NC21
NC22
C10
NC24
NC25
NC27
C13
NC30
NC31
C18
NC32
NC35
NC37
NC40
NC41
C24
NC46
NC49
NC57
NC58
C29
NC62
NC67
NC69
NC73
NC74
NC78
NC80
NC82
NC84
C37
NC85
NC86
C39
NC88
NC93
NC94
NC95
C43
NC96
C44
NC97
NC99
NC100
C49
NC101
C53
C55
NC105
C1
NC1
C3
NC2
NC15
C7
NC19
NC20
C8
C9
C10
C12
NC25
NC26
NC27
NC28
NC31
C17
NC34
NC35
NC37
C20
NC41
NC42
C22
NC46
NC57
NC58
C28
C29
NC60
C30
NC61
NC64
NC66
NC67
NC71
NC80
NC81
NC82
C37
C38
NC86
NC87
NC88
C40
NC89
NC91
NC93
NC94
NC95
C45
C46
C47
NC99
NC101
NC102
C53
NC103
NC105
C1
NC1
NC2
NC6
NC9
C5
NC14
C6
C7
C8
C9
C11
C13
C14
NC28
NC30
C18
NC35
NC36
NC37
C20
NC39
NC41
C23
C25
NC45
NC46
NC48
NC53
NC54
NC59
C28
C30
NC67
NC69
NC71
NC73
NC75
NC78
NC79
NC80
C35
C36
NC8

In [1344]:
# make an average
finaldf = tmp[0] # just to get the format 
for i in range(0, 11): # clear the format
    finaldf.iat[i,0] = 0
for j in range(0, len(tmp)): # fill with the total
    for i in range(0, 11):
        finaldf.iat[i,0] += tmp[j].iat[i, 0]

for i in range(0, 11): # divide it out for the avg
    finaldf.iat[i,0] = finaldf.iat[i, 0] / len(tmp)
    
finaldf
        

,Values
TP,1.36
FP,0.0
TN,38.48
FN,18.64
Accuracy,0.654538
Sensitivity,0.067622
Specificity,0.96
Missrate,0.892378
Precision,0.72
FDR,0.0


In [1338]:
tmpdf = df.columns
tmpdf2 = hw9df.columns
tmpdf1 = set(tmpdf) - set(tmpdf2)
print(tmpdf1)

{'CSNK1G3_GRCh37_5:122926143-122926144_Frame-Shift-Ins_INS_----T', 'HOXA7_GRCh37_7:27194754-27194754_Missense-Mutation_SNP_G-G-A', 'CTNNB1_GRCh37_3:41266103-41266103_Missense-Mutation_SNP_G-G-A', 'IFNL3_GRCh37_19:39734787-39734787_Missense-Mutation_SNP_C-C-T', 'CELA3A_GRCh37_1:22336257-22336257_Silent_SNP_T-T-C', "STC2_GRCh37_5:172744700-172744700_3'UTR_SNP_G-G-A", 'TRGJP_GRCh37_7:38313227-38313227_Frame-Shift-Del_DEL_T-T--', 'TNFRSF25_GRCh37_1:6525249-6525249_Missense-Mutation_SNP_C-C-T', 'TIMP3_GRCh37_22:33255323-33255324_Frame-Shift-Ins_INS_----C', 'ZUFSP_GRCh37_6:116967058-116967058_Frame-Shift-Del_DEL_T-T--', 'LZTS2_GRCh37_10:102763509-102763509_Silent_SNP_C-C-T', 'LIPT1_GRCh37_2:99778504-99778504_Frame-Shift-Del_DEL_A-A--', 'SERPINE2_GRCh37_2:224866442-224866442_Missense-Mutation_SNP_G-G-A', 'BUB1B_GRCh37_15:40512803-40512803_Missense-Mutation_SNP_G-G-A', 'YLPM1_GRCh37_14:75230471-75230471_Frame-Shift-Del_DEL_G-G--', 'MTUS2_GRCh37_13:29933476-29933476_Missense-Mutation_SNP_C-C-T'

In [1475]:
############################################ HW 10 ##########################################################################
hw10df = df.iloc[0:163, 0:1411]
ffdf1 = hw10df
hw10df

,Unnamed: 0,FARP1_GRCh37_13:99092237-99092237_Frame-Shift-Del_DEL_G-G--,QKI_GRCh37_6:163987695-163987695_Intron_DEL_T-T--,NHLRC2_GRCh37_10:115662308-115662308_Frame-Shift-Del_DEL_A-A--,PDZD8_GRCh37_10:119042898-119042898_Frame-Shift-Del_DEL_T-T--,EBF3_GRCh37_10:131666125-131666125_Frame-Shift-Del_DEL_C-C--,MASTL_GRCh37_10:27459272-27459272_Frame-Shift-Del_DEL_A-A--,PTCHD3_GRCh37_10:27687763-27687763_Frame-Shift-Del_DEL_A-A--,PFKP_GRCh37_10:3151612-3151612_Silent_SNP_C-C-T,CREM_GRCh37_10:35456559-35456559_Intron_DEL_T-T--,...,PHF2_GRCh37_9:96422621-96422621_Missense-Mutation_SNP_A-A-G,PHF2_GRCh37_9:96422642-96422642_Missense-Mutation_SNP_C-C-A,USP51_GRCh37_X:55513209-55513209_3'UTR_DEL_T-T--,CDH23_GRCh37_10:73405660-73405660_Missense-Mutation_SNP_G-G-A,SLC26A11_GRCh37_17:78195370-78195370_Missense-Mutation_SNP_C-C-T,MME_GRCh37_3:154866382-154866382_Missense-Mutation_SNP_A-A-T_A-A-C,CADPS_GRCh37_3:62459960-62459960_Missense-Mutation_SNP_G-G-T,SCN4A_GRCh37_17:62049998-62049998_Silent_SNP_G-G-A,FPR2_GRCh37_19:52272624-52272624_Missense-Mutation_SNP_G-G-A,OLFM3_GRCh37_1:102302454-102302454_Missense-Mutation_SNP_A-A-C_A-A-G
0,C1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NC1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,C3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NC2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,NC104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159,NC105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
160,NC106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
161,C56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [1476]:
############################################ Part 1 ##########################################################################
# go through and only get features that are in >= 4 samples
dellist = []
for i in range(1, ffdf1.shape[1]):
    if(ffdf1.iloc[162, i] < 4):
        dellist.append(i)
# end of for

# delete the list
ffdf1.drop(ffdf1.columns[dellist], axis=1,inplace=True)
ffdf1 = ffdf1.drop(162)
# output
ffdf1

,Unnamed: 0,QKI_GRCh37_6:163987695-163987695_Intron_DEL_T-T--,EBF3_GRCh37_10:131666125-131666125_Frame-Shift-Del_DEL_C-C--,OR5M3_GRCh37_11:56237709-56237709_Frame-Shift-Del_DEL_T-T--,MVK_GRCh37_12:110019240-110019240_Frame-Shift-Del_DEL_C-C--,ZFC3H1_GRCh37_12:72038781-72038781_Frame-Shift-Del_DEL_T-T--,TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_C-C-T,NLK_GRCh37_17:26449740-26449740_Frame-Shift-Del_DEL_T-T--,KLHL14_GRCh37_18:30349862-30349862_Frame-Shift-Del_DEL_G-G--,PLEKHA6_GRCh37_1:204228411-204228411_Frame-Shift-Del_DEL_C-C--,...,SCN2B_GRCh37_11:118047235-118047235_5'UTR_DEL_A-A--,MYCN_GRCh37_2:16082314-16082314_Frame-Shift-Del_DEL_C-C--,CNTN6_GRCh37_3:1371501-1371501_Frame-Shift-Del_DEL_A-A--,TCERG1_GRCh37_5:145886723-145886723_Frame-Shift-Del_DEL_A-A--,TRIP11_GRCh37_14:92472698-92472698_Frame-Shift-Del_DEL_T-T--,PRPF4B_GRCh37_6:4031881-4031881_Frame-Shift-Del_DEL_A-A--,STAMBPL1_GRCh37_10:90682146-90682146_Frame-Shift-Del_DEL_A-A--,CDH24_GRCh37_14:23517514-23517514_Frame-Shift-Del_DEL_G-G--,PTENP1_GRCh37_9:33675379-33675379_RNA_SNP_C-C-A,EDNRB_GRCh37_13:78475267-78475267_Frame-Shift-Del_DEL_A-A--
0,C1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NC1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,C3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NC2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,C55,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
158,NC104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
159,NC105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,NC106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1492]:
# ############################################ Part 2 ##########################################################################
# Part A
tuplelist = []
ffdf2 = ffdf1
# get the tuples
for i in range(1, ffdf2.shape[1]):
    for j in range(1, ffdf2.shape[1]):
        col2 = i + j + 1
        if col2 < ffdf2.shape[1]:
            counter = 0
            for k in range(0, ffdf2.shape[0]):
                if ffdf2.iloc[k, i] == 1 and ffdf2.iloc[k, col2] == 1:
                    counter += 1
            if counter >= 4:
                tup = (ffdf2.columns[i], ffdf2.columns[col2], counter)
                tuplelist.append(tup)

print(len(tuplelist))        

61


In [1893]:
# Part b - d
hw10df2 = pd.DataFrame(columns=['Association', 'Support', 'Confidence', 'SupportxConfidence'])
for i in range(0, len(tuplelist)):
    # calc confidence, need to see how many times tuplelist[i][0] is in the list
    count = 0
    for j in range(0, len(tuplelist)):
        if tuplelist[j][0] == tuplelist[i][0]:
            count += 1
            
    # change this to the counter for support and confidence, 122 -> 162
    hw10df2.loc[len(hw10df2.index)] = [tuplelist[i][0]+'->'+tuplelist[i][1],2/122, 2 / count, (2/122) * (2/count)]
    
    # for the reverse
    count = 0
    for j in range(0, len(tuplelist)):
        if tuplelist[j][1] == tuplelist[i][1]:
            count += 1
    hw10df2.loc[len(hw10df2.index)] = [tuplelist[i][1]+'->'+tuplelist[i][0],2/122, 2 / count, (2/122) * (2/count)]

hw10df2

'TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_C-C-T->RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del_DEL_T-T--'

In [1521]:
# top amount of support
hw10df2

,Association,Support,Confidence,SupportxConfidence
0,QKI_GRCh37_6:163987695-163987695_Intron_DEL_T-...,0.016393,2.000000,0.032787
91,RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del...,0.016393,0.200000,0.003279
89,AKAP7_GRCh37_6:131481276-131481276_Frame-Shift...,0.016393,1.000000,0.016393
88,RNF43_GRCh37_17:56435161-56435161_Frame-Shift-...,0.016393,0.666667,0.010929
87,PGM5_GRCh37_9:70993145-70993145_Missense-Mutat...,0.016393,0.400000,0.006557
...,...,...,...,...
36,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.016393,0.181818,0.002981
35,DONSON_GRCh37_21:34947889-34947889_3'Flank_DEL...,0.016393,1.000000,0.016393
34,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.016393,0.181818,0.002981
33,RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del...,0.016393,0.200000,0.003279


In [1522]:
# top amount of confidence
tmpdf = hw10df2.sort_values(by='Confidence',ascending=False)
tmpdf

,Association,Support,Confidence,SupportxConfidence
0,QKI_GRCh37_6:163987695-163987695_Intron_DEL_T-...,0.016393,2.000000,0.032787
19,ZC3H18_GRCh37_16:88691141-88691141_Frame-Shift...,0.016393,2.000000,0.032787
29,KDM1B_GRCh37_6:18222307-18222307_3'UTR_DEL_A-A...,0.016393,2.000000,0.032787
43,CCAR2_GRCh37_8:22472975-22472975_Frame-Shift-D...,0.016393,2.000000,0.032787
45,LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del...,0.016393,2.000000,0.032787
...,...,...,...,...
58,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-D...,0.016393,0.181818,0.002981
60,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-D...,0.016393,0.181818,0.002981
62,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-D...,0.016393,0.181818,0.002981
20,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.016393,0.181818,0.002981


In [1707]:
# min threshhold for confidence, .5
tmpdf = hw10df2
dellater = []
# go through a delete rows with confidence <= .5
for i in range(0, tmpdf.shape[0]):
    if tmpdf.iloc[i, 2] <= .5:
        dellater.append(i)
# end of i

# delete the list
tmpdf = tmpdf.drop(dellater)
tmpdf
del dellater

,Association,Support,Confidence,SupportxConfidence
0,QKI_GRCh37_6:163987695-163987695_Intron_DEL_T-...,0.016393,2.000000,0.032787
43,CCAR2_GRCh37_8:22472975-22472975_Frame-Shift-D...,0.016393,2.000000,0.032787
120,MXRA8_GRCh37_1:1290110-1290110_Frame-Shift-Del...,0.016393,2.000000,0.032787
115,TVP23A_GRCh37_16:10867203-10867203_Frame-Shift...,0.016393,2.000000,0.032787
112,C11orf35_GRCh37_11:558069-558069_RNA_DEL_G-G--...,0.016393,2.000000,0.032787
110,AKAP7_GRCh37_6:131481276-131481276_Frame-Shift...,0.016393,2.000000,0.032787
108,UBR5_GRCh37_8:103289349-103289349_Frame-Shift-...,0.016393,2.000000,0.032787
105,RABGAP1_GRCh37_9:125861042-125861042_Frame-Shi...,0.016393,2.000000,0.032787
103,LHX1_GRCh37_17:35300395-35300395_Frame-Shift-D...,0.016393,2.000000,0.032787
101,UPF3A_GRCh37_13:115057211-115057211_Frame-Shif...,0.016393,2.000000,0.032787


In [1519]:
# min threshhold for support (static doesnt matter)
hw10df2

,Association,Support,Confidence,SupportxConfidence
0,QKI_GRCh37_6:163987695-163987695_Intron_DEL_T-...,0.016393,2.000000,0.032787
1,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.016393,1.000000,0.016393
2,OR5M3_GRCh37_11:56237709-56237709_Frame-Shift-...,0.016393,2.000000,0.032787
3,ACVR2A_GRCh37_2:148683686-148683686_Frame-Shif...,0.016393,0.500000,0.008197
4,TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_...,0.016393,0.400000,0.006557
...,...,...,...,...
117,DONSON_GRCh37_21:34947889-34947889_3'Flank_DEL...,0.016393,1.000000,0.016393
118,RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del...,0.016393,0.666667,0.010929
119,CDH24_GRCh37_14:23517514-23517514_Frame-Shift-...,0.016393,1.000000,0.016393
120,MXRA8_GRCh37_1:1290110-1290110_Frame-Shift-Del...,0.016393,2.000000,0.032787


In [1704]:
# Show the top of support * confidence
tmpdf = hw10df2.sort_values(by='SupportxConfidence',ascending=False)
tmpdf

,Association,Support,Confidence,SupportxConfidence
0,QKI_GRCh37_6:163987695-163987695_Intron_DEL_T-...,0.016393,2.000000,0.032787
19,ZC3H18_GRCh37_16:88691141-88691141_Frame-Shift...,0.016393,2.000000,0.032787
29,KDM1B_GRCh37_6:18222307-18222307_3'UTR_DEL_A-A...,0.016393,2.000000,0.032787
43,CCAR2_GRCh37_8:22472975-22472975_Frame-Shift-D...,0.016393,2.000000,0.032787
45,LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del...,0.016393,2.000000,0.032787
...,...,...,...,...
58,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-D...,0.016393,0.181818,0.002981
60,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-D...,0.016393,0.181818,0.002981
62,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-D...,0.016393,0.181818,0.002981
20,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.016393,0.181818,0.002981


In [1658]:
# ############################################ Part 3 ##########################################################################
# Take 2 pairs that have 1 feature in common and add the 2 that are part of the pairs
# triplelist = []
# tuplelist
for i in range(0, len(tuplelist)):
    for j in range(1, len(tuplelist)):
        tuplelist[i].isin(tuplelist[j]):
            print("its in this list")


In [1661]:
#  make all permutations of the triples A,B A,C B,C
allperm = []
for i in range(0, len(triplelist)):
    allperm.append((triplelist[i][0], triplelist[i][1], triplelist[i][3]))
    allperm.append((triplelist[i][0], triplelist[i][2], triplelist[i][3]))
    allperm.append((triplelist[i][1], triplelist[i][2], triplelist[i][3]))

# go through each pair and see if they are in the same patients >= 4 times
dellater = []
for i in range(0, len(allperm)):
    x = allperm[i][0]
    y = allperm[i][1]
    counter = 0
    for j in range(0, ffdf1.shape[0]):
        if ffdf1.loc[j, x] == 1 and ffdf1.loc[j, y] == 1:
            counter += 1
    if counter >= 4:
        dellater.append(allperm[i][2])

tmp = []
for i in range(0, (len(dellater) - 3)):
    if dellater[i] == dellater[i + 1] and dellater[i] == dellater[i + 2]:
        tmp.append(dellater[i])

del dellater

In [1663]:
tmp1111 = []
for i in range(0, len(triplelist)):
    if triplelist[i][3] in tmp:
        tmp1111.append(triplelist[i])
        
triplelist = tmp1111
del tmp1111
del tmp

In [1673]:
# Part B
hw10df3 = pd.DataFrame(columns=['Association', 'Support', 'Confidence', 'SupportxConfidence'])
for i in range(len(triplelist)):
    ab = 0
    ac = 0
    bc = 0
    b = 0
    c = 0
    a = 0
    for j in range(len(triplelist)):
        if triplelist[i][0] in triplelist[j] and triplelist[i][1] in triplelist[j] and j != i:
            ab += 1
        if triplelist[i][0] in triplelist[j] and triplelist[i][2] in triplelist[j] and j != i:
            ac += 1
        if triplelist[i][1] in triplelist[j] and triplelist[i][2] in triplelist[j] and j != i:
            bc += 1
        if triplelist[i][0] in triplelist[j] and j != i:
            a += 1
        if triplelist[i][1] in triplelist[j] and j != i:
            b += 1
        if triplelist[i][2] in triplelist[j] and j != i:
            c += 1
    # end of j
    # ab
    hw10df3.loc[(len(hw10df3.index))] = [triplelist[i][0]+'and'+triplelist[i][1]+'->'+triplelist[i][2], ab/len(tuplelist), (ab/len(tuplelist)) / a, (ab/len(tuplelist) * (ab/len(tuplelist)) / a)]
    # ac
    hw10df3.loc[(len(hw10df3.index))] = [triplelist[i][0]+'and'+triplelist[i][2]+'->'+triplelist[i][1], ac/len(tuplelist), (ac/len(tuplelist)) / a, (ab/len(tuplelist) * (ab/len(tuplelist)) / a)]
    # bc
    hw10df3.loc[(len(hw10df3.index))] = [triplelist[i][1]+'and'+triplelist[i][2]+'->'+triplelist[i][0], bc/len(tuplelist), (bc/len(tuplelist)) / b, (ab/len(tuplelist) * (ab/len(tuplelist)) / b)]
    # ba
    hw10df3.loc[(len(hw10df3.index))] = [triplelist[i][1]+'and'+triplelist[i][0]+'->'+triplelist[i][2], ab/len(tuplelist), (ab/len(tuplelist)) / b, (ab/len(tuplelist) * (ab/len(tuplelist)) / b)]
    # ca
    hw10df3.loc[(len(hw10df3.index))] = [triplelist[i][2]+'and'+triplelist[i][0]+'->'+triplelist[i][1], ac/len(tuplelist), (ac/len(tuplelist)) / c, (ac/len(tuplelist) * (ac/len(tuplelist)) / c)]
    # cb
    hw10df3.loc[(len(hw10df3.index))] = [triplelist[i][2]+'and'+triplelist[i][1]+'->'+triplelist[i][0], bc/len(tuplelist), (bc/len(tuplelist)) / c, (bc/len(tuplelist) * (bc/len(tuplelist)) / c)]

In [1886]:
# output top support
hw10df3.sort_values('Support', ascending=False).head(50)

,Association,Support,Confidence,SupportxConfidence
276,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.213115,0.005760,0.001228
273,RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del...,0.213115,0.005464,0.001165
101,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.213115,0.005760,0.001228
279,RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del...,0.213115,0.005464,0.001165
196,ACVR2A_GRCh37_2:148683686-148683686_Frame-Shif...,0.213115,0.005760,0.001228
282,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.213115,0.005760,0.001228
285,RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del...,0.213115,0.005464,0.001165
289,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.213115,0.005760,0.000007
292,RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del...,0.213115,0.005464,0.001165
295,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.213115,0.005760,0.000007


In [1887]:
# output top confidence
hw10df3.sort_values('Confidence', ascending=False).head(50)

,Association,Support,Confidence,SupportxConfidence
184,POLM_GRCh37_7:44120414-44120414_Frame-Shift-De...,0.016393,0.016393,0.000269
300,PIK3CA_GRCh37_3:178952085-178952085_Missense-M...,0.016393,0.016393,0.000269
185,POLM_GRCh37_7:44120414-44120414_Frame-Shift-De...,0.016393,0.016393,0.000269
307,PIK3CA_GRCh37_3:178952085-178952085_Missense-M...,0.016393,0.016393,0.000269
306,PIK3CA_GRCh37_3:178952085-178952085_Missense-M...,0.016393,0.016393,0.000269
301,PIK3CA_GRCh37_3:178952085-178952085_Missense-M...,0.016393,0.016393,0.000269
520,CDH24_GRCh37_14:23517514-23517514_Frame-Shift-...,0.016393,0.016393,0.000269
524,CDH24_GRCh37_14:23517514-23517514_Frame-Shift-...,0.016393,0.016393,0.000269
525,CDH24_GRCh37_14:23517514-23517514_Frame-Shift-...,0.016393,0.016393,0.000269
521,CDH24_GRCh37_14:23517514-23517514_Frame-Shift-...,0.016393,0.016393,0.000269


In [1687]:
# Threshold for support .2
tmpdf = hw10df3
dellater = []
for i in range(0, tmpdf.shape[0]):
    if tmpdf.iloc[i, 1] <= .2:
        dellater.append(i)

tmpdf = tmpdf.drop(dellater)
del dellater
tmpdf

,Association,Support,Confidence,SupportxConfidence
8,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.213115,0.005760,0.000356
11,ACVR2A_GRCh37_2:148683686-148683686_Frame-Shif...,0.213115,0.005760,0.001228
20,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.213115,0.005760,0.000356
23,RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del...,0.213115,0.005464,0.001165
50,ACVR2A_GRCh37_2:148683686-148683686_Frame-Shif...,0.213115,0.005760,0.000356
...,...,...,...,...
513,RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del...,0.213115,0.005464,0.001165
516,ACVR2A_GRCh37_2:148683686-148683686_Frame-Shif...,0.213115,0.005760,0.001228
519,RPL22_GRCh37_1:6257785-6257785_Frame-Shift-Del...,0.213115,0.005464,0.001165
523,ACVR2A_GRCh37_2:148683686-148683686_Frame-Shif...,0.213115,0.005760,0.000007


In [1696]:
# threshold for confidence .009
tmpdf = hw10df3
dellater = []
for i in range(0, tmpdf.shape[0]):
    if tmpdf.iloc[i, 2] <= .009:
        dellater.append(i)

tmpdf = tmpdf.drop(dellater)
del dellater
tmpdf

,Association,Support,Confidence,SupportxConfidence
130,PHF2_GRCh37_9:96422612-96422612_Frame-Shift-De...,0.049180,0.009836,0.000484
131,PHF2_GRCh37_9:96422612-96422612_Frame-Shift-De...,0.049180,0.009836,0.000484
160,PHF2_GRCh37_9:96422612-96422612_Frame-Shift-De...,0.049180,0.009836,0.000484
161,PHF2_GRCh37_9:96422612-96422612_Frame-Shift-De...,0.049180,0.009836,0.000484
184,POLM_GRCh37_7:44120414-44120414_Frame-Shift-De...,0.016393,0.016393,0.000269
...,...,...,...,...
539,AKAP7_GRCh37_6:131481276-131481276_Frame-Shift...,0.049180,0.009836,0.000484
540,KDM1B_GRCh37_6:18222307-18222307_3'UTR_DEL_A-A...,0.049180,0.009836,0.000484
541,KDM1B_GRCh37_6:18222307-18222307_3'UTR_DEL_A-A...,0.049180,0.009836,0.000484
546,KDM1B_GRCh37_6:18222307-18222307_3'UTR_DEL_A-A...,0.049180,0.009836,0.000484


In [1885]:
# output the threshold top supportxconfidence
tmpdf = hw10df3
dellater = []
for i in range(0, tmpdf.shape[0]):
    if tmpdf.iloc[i, 2] <= .009 or tmpdf.iloc[i, 1] <= .02:
        dellater.append(i)

tmpdf = tmpdf.drop(dellater)
tmp1df = tmpdf.reset_index(drop=True)
del dellater
tmp1df.sort_values('SupportxConfidence', ascending=False)

,Association,Support,Confidence,SupportxConfidence
21,LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del...,0.081967,0.009107,0.000747
19,PGM5_GRCh37_9:70993145-70993145_Missense-Mutat...,0.081967,0.009107,0.000747
43,PGM5_GRCh37_9:70993145-70993145_Missense-Mutat...,0.081967,0.009107,0.000747
55,PGM5_GRCh37_9:70993145-70993145_Missense-Mutat...,0.081967,0.009107,0.000747
28,LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del...,0.081967,0.009107,0.000747
...,...,...,...,...
39,KIAA0195_GRCh37_17:73491063-73491063_Frame-Shi...,0.049180,0.009836,0.000484
41,KIAA0195_GRCh37_17:73491063-73491063_Frame-Shi...,0.049180,0.009836,0.000484
71,KDM1B_GRCh37_6:18222307-18222307_3'UTR_DEL_A-A...,0.049180,0.009836,0.000484
44,LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del...,0.081967,0.009107,0.000269


In [1888]:
hw10df3.sort_values('SupportxConfidence',ascending=False).head(50)

,Association,Support,Confidence,SupportxConfidence
551,UBR5_GRCh37_8:103289349-103289349_Frame-Shift-...,0.180328,0.007213,0.001301
509,UBR5_GRCh37_8:103289349-103289349_Frame-Shift-...,0.180328,0.007213,0.001301
311,UBR5_GRCh37_8:103289349-103289349_Frame-Shift-...,0.180328,0.007213,0.001301
431,UBR5_GRCh37_8:103289349-103289349_Frame-Shift-...,0.180328,0.007213,0.001301
281,UBR5_GRCh37_8:103289349-103289349_Frame-Shift-...,0.180328,0.007213,0.001301
119,UBR5_GRCh37_8:103289349-103289349_Frame-Shift-...,0.180328,0.007213,0.001301
177,ACVR2A_GRCh37_2:148683686-148683686_Frame-Shif...,0.213115,0.005760,0.001228
180,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.213115,0.005760,0.001228
176,ACVR2A_GRCh37_2:148683686-148683686_Frame-Shif...,0.213115,0.005760,0.001228
175,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.213115,0.005760,0.001228


In [398]:
############################################ HW 11 ##########################################################################
hw11df = df.iloc[0:163, 0:1411]
# go through and delete mutations not in at least 5 patients and delete non-cancer patients
delrow = []
for i in range(hw11df.shape[0] - 1):
    if hw11df.iloc[i, 0].find('NC') == 0:
        delrow.append(i)

hw11df = hw11df.drop(delrow)
hw11df = hw11df.reset_index(drop=True)
# del delrow
delcol = []
for i in range(1, hw11df.shape[1]):
    if hw11df.iloc[hw11df.shape[0] - 1, i] < 3.0:
        delcol.append(i) 
        
hw11df = hw11df.drop(hw11df.columns[delcol], axis=1)
hw11df = hw11df.drop(hw11df.shape[0] - 1)
del delcol

hw11df

,Unnamed: 0,QKI_GRCh37_6:163987695-163987695_Intron_DEL_T-T--,EBF3_GRCh37_10:131666125-131666125_Frame-Shift-Del_DEL_C-C--,PTCHD3_GRCh37_10:27687763-27687763_Frame-Shift-Del_DEL_A-A--,PTEN_GRCh37_10:89720812-89720812_Frame-Shift-Del_DEL_A-A--,OR5M3_GRCh37_11:56237709-56237709_Frame-Shift-Del_DEL_T-T--,MVK_GRCh37_12:110019240-110019240_Frame-Shift-Del_DEL_C-C--,ZFC3H1_GRCh37_12:72038781-72038781_Frame-Shift-Del_DEL_T-T--,PSMB5_GRCh37_14:23495175-23495175_3'UTR_DEL_A-A--,DDHD1_GRCh37_14:53513450-53513480_3'UTR_DEL_AATCAGTTTTAGGCCATTCATGTCCTTCAAG-AATCAGTTTTAGGCCATTCATGTCCTTCAAG-TCAGTTTTAGGCCATTCATGTCCTTCA,...,WDR55_GRCh37_5:140049102-140049102_Frame-Shift-Del_DEL_A-A--,CBX5_GRCh37_12:54645832-54645832_Frame-Shift-Del_DEL_T-T--,GPR171_GRCh37_3:150916134-150916134_3'UTR_SNP_T-T-G,GPR171_GRCh37_3:150917110-150917110_Frame-Shift-Del_DEL_A-A--,TIMP3_GRCh37_22:33255323-33255324_Frame-Shift-Ins_INS_----C,FUS_GRCh37_16:31196405-31196405_Silent_SNP_C-C-T,NELFE_GRCh37_6:31926206-31926206_Frame-Shift-Del_DEL_G-G--,DDX10_GRCh37_11:108544239-108544239_Frame-Shift-Del_DEL_A-A--,MYO18A_GRCh37_17:27493332-27493332_Frame-Shift-Del_DEL_G-G--,FASTKD1_GRCh37_2:170428529-170428530_Frame-Shift-Ins_INS_----T
0,C1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,C3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,C4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,C5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,C6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,C8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,C9,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,C10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [404]:
# make F2
dfFN = [] # list of dataframes of FN's
pairs = []
for i in range(1, hw11df.shape[1]):
    for j in range(1, hw11df.shape[1]):
        col2 = i + j + 1
        if col2 < hw11df.shape[1]:
            counter = 0
            for k in range(0, hw11df.shape[0]):
                if hw11df.iloc[k, i] == 1 and hw11df.iloc[k, col2] == 1:
                    counter += 1
            if counter >= 3:
                tup = (hw11df.columns[i], hw11df.columns[col2])
                pairs.append(tup)

dfFN.append(pairs) # a list of a list of pairs
print(len(dfFN[0]))
del pairs


402


In [405]:
# make f3
triplets = []
for i in range(len(dfFN[0])):
    for j in range(i + 1, len(dfFN[0])):
        if dfFN[0][i][0] == dfFN[0][j][0] and dfFN[0][i][1] != dfFN[0][j][1]:
            counter = 0
            for k in range(hw11df.shape[0]):
                x = dfFN[0][i][0]
                y = dfFN[0][i][1]
                z = dfFN[0][j][1]
                if hw11df.loc[k, x] == 1 and hw11df.loc[k, y] == 1 and hw11df.loc[k, z] == 1:
                    counter += 1
            if counter >= 3:
                # print('\n')
                # print(dfFN[0][i], dfFN[0][j])
                trip = (dfFN[0][i][0], dfFN[0][i][1], dfFN[0][j][1])
                triplets.append(trip)
if len(triplets) > 0:
    dfFN.append(triplets)
    print(len(dfFN[1]))

# print(dfFN[1])


225


In [406]:
# make f4
quad = []
for i in range(len(dfFN[1])):
    for j in range(i + 1, len(dfFN[1])):
        if dfFN[1][i][0] == dfFN[1][j][0] and dfFN[1][i][1] == dfFN[1][j][1] and dfFN[1][i][2] != dfFN[1][j][2]:
            counter = 0
            for k in range(hw11df.shape[0]):
                x = dfFN[1][i][0]
                y = dfFN[1][i][1]
                z = dfFN[1][i][2]
                a = dfFN[1][j][2]
                if hw11df.loc[k, x] == 1 and hw11df.loc[k, y] == 1 and hw11df.loc[k, z] == 1 and hw11df.loc[k, a] == 1:
                    counter += 1
            if counter >= 3:
                qua = (dfFN[1][i][0],dfFN[1][i][1], dfFN[1][i][2], dfFN[1][j][2])
                # print(dfFN[1][i][0],dfFN[1][i][1], dfFN[1][i][2], dfFN[1][j][2])
                quad.append(qua)
                # print(qua)
if len(quad) > 0:
    dfFN.append(quad)
    # print(dfFN[2])
    print(len(dfFN[2]))


60


In [407]:
# make f5
five = []
for i in range(len(dfFN[2])):
    for j in range(i + 1, len(dfFN[2])):
        if dfFN[2][i][0] == dfFN[2][j][0] and dfFN[2][i][1] == dfFN[2][j][1] and dfFN[2][i][2] == dfFN[2][j][2] and dfFN[2][i][3] != dfFN[2][j][3]:
            counter = 0
            for k in range(hw11df.shape[0]):
                x = dfFN[2][i][0]
                y = dfFN[2][i][1]
                z = dfFN[2][i][2]
                a = dfFN[2][i][3]
                b = dfFN[2][j][3]
                if hw11df.loc[k, x] == 1 and hw11df.loc[k, y] == 1 and hw11df.loc[k, z] == 1 and hw11df.loc[k, a] == 1 and hw11df.loc[k,b] == 1:
                    counter += 1
            if counter >= 3:
                fiveset = (dfFN[2][i][0],dfFN[2][i][1], dfFN[2][i][2], dfFN[2][i][2], dfFN[2][j][3])
                five.append(fiveset)
if len(five) > 0:
    dfFN.append(five)
    print(len(dfFN[3]))


8


In [408]:
# go through the dfFN and make dataframes for them with rules, support, confidence, supportxconfidence
# for f2
dff2 = pd.DataFrame(columns=['Association', 'Support', 'Confidence', 'SupportxConfidence'])
dff3 = pd.DataFrame(columns=['Association', 'Support', 'Confidence', 'SupportxConfidence'])
dff4 = pd.DataFrame(columns=['Association', 'Support', 'Confidence', 'SupportxConfidence'])
dff5 = pd.DataFrame(columns=['Association', 'Support', 'Confidence', 'SupportxConfidence'])
#  f2
for i in range(len(dfFN[0])):
    # get how many times that features is in a patients
    acount, feature_count = 0, 0
    for j in range(hw11df.shape[0]):
        if hw11df.loc[j, dfFN[0][i][0]] == 1 and hw11df.loc[j, dfFN[0][i][1]] == 1:
            feature_count += 1
        if hw11df.loc[j, dfFN[0][i][0]] == 1:
            acount += 1
    dff2.loc[len(dff2.index)] = [dfFN[0][i][0]+'->'+dfFN[0][i][1], feature_count/56, feature_count/acount, feature_count/56 * feature_count/acount]
    
# f3
for i in range(len(dfFN[1])):
    # get how many times that features is in a patients
    acount, feature_count = 0, 0
    for j in range(hw11df.shape[0]):
        if hw11df.loc[j, dfFN[1][i][0]] == 1 and hw11df.loc[j, dfFN[1][i][1]] == 1 and hw11df.loc[j, dfFN[1][i][2]] == 1:
            feature_count += 1
        if hw11df.loc[j, dfFN[1][i][0]] == 1 and hw11df.loc[j, dfFN[1][i][1]] == 1:
            acount += 1
    dff3.loc[len(dff3.index)] = [dfFN[1][i][0]+'and'+dfFN[1][i][1]+'->'+dfFN[1][i][2], feature_count/56, feature_count/acount, feature_count/56 * feature_count/acount]
    
# f4
for i in range(len(dfFN[2])):
    # get how many times that features is in a patients
    acount, feature_count = 0, 0
    for j in range(hw11df.shape[0]):
        if hw11df.loc[j, dfFN[2][i][0]] == 1 and hw11df.loc[j, dfFN[2][i][1]] == 1 and hw11df.loc[j, dfFN[2][i][2]] == 1 and hw11df.loc[j, dfFN[2][i][3]] == 1:
            feature_count += 1
        if hw11df.loc[j, dfFN[2][i][0]] == 1 and hw11df.loc[j, dfFN[2][i][1]] == 1 and hw11df.loc[j, dfFN[2][i][2]] == 1:
            acount += 1
    dff4.loc[len(dff4.index)] = [dfFN[2][i][0]+'and'+dfFN[2][i][1]+'and'+dfFN[2][i][2]+'->'+dfFN[2][i][3], feature_count/56, feature_count/acount, feature_count/56 * feature_count/acount]
 
# f5
for i in range(len(dfFN[3])):
    # get how many times that features is in a patients
    acount, feature_count = 0, 0
    for j in range(hw11df.shape[0]):
        if hw11df.loc[j, dfFN[3][i][0]] == 1 and hw11df.loc[j, dfFN[3][i][1]] == 1 and hw11df.loc[j, dfFN[3][i][2]] == 1 and hw11df.loc[j, dfFN[3][i][3]] == 1 and hw11df.loc[j, dfFN[3][i][4]] == 1:
            feature_count += 1
        if hw11df.loc[j, dfFN[3][i][0]] == 1 and hw11df.loc[j, dfFN[3][i][1]] == 1 and hw11df.loc[j, dfFN[3][i][2]] == 1 and hw11df.loc[j, dfFN[3][i][3]] == 1:
            acount += 1
    dff5.loc[len(dff5.index)] = [dfFN[3][i][0]+'and'+dfFN[3][i][1]+'and'+dfFN[3][i][2]+'and'+dfFN[3][i][3]+'->'+dfFN[3][i][4], feature_count/56, feature_count/acount, feature_count/56 * feature_count/acount]
     

,Association,Support,Confidence,SupportxConfidence
0,TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_...,0.018519,1.00,0.018519
1,TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_...,0.018519,0.75,0.013889
2,TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_...,0.018519,1.00,0.018519
3,TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_...,0.018519,0.75,0.013889
4,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.018519,1.00,0.018519
5,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.018519,1.00,0.018519
6,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-D...,0.018519,0.75,0.013889
7,LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del...,0.018519,1.00,0.018519


In [376]:
# make f6
six = []
for i in range(len(dfFN[3])):
    for j in range(i + 1, len(dfFN[3])):
        if dfFN[3][i][0] == dfFN[3][j][0] and dfFN[3][i][1] == dfFN[3][j][1] and dfFN[3][i][2] == dfFN[3][j][2] and dfFN[3][i][3] == dfFN[3][j][3] and dfFN[3][i][4] != dfFN[3][j][4]:
            counter = 0
            for k in range(hw11df.shape[0]):
                x = dfFN[3][i][0]
                y = dfFN[3][i][1]
                z = dfFN[3][i][2]
                a = dfFN[3][i][3]
                b = dfFN[3][i][3]
                c = dfFN[3][j][4]
                if hw11df.loc[k, x] == 1 and hw11df.loc[k, y] == 1 and hw11df.loc[k, z] == 1 and hw11df.loc[k, a] == 1 and hw11df.loc[k,b] == 1 and hw11df.loc[k,c] == 1:
                    counter += 1
            if counter >= 3:
                sixset = (dfFN[3][i][0],dfFN[3][i][1], dfFN[3][i][2], dfFN[3][i][2], dfFN[3][i][3], dfFN[3][j][4])
                six.append(fiveset)
if len(six) > 0:
    dfFN.append(six)
    print(len(dfFN[4]))


In [300]:
def get_feature(last_list, lastsize):    
    # go through the last list and match it by lastsize - 1
    current_list = []
    # ll = []    
    for i in range(0, len(last_list)): # go through the list
        for k in range(1, len(last_list)):
            tmp = ()
            if last_list[i][0] == last_list[k][0]:
                # for j in range(0, lastsize - 2): # go through the elements in the list
                j = 0
                while(j <= lastsize - 2):
                    # check if pos A to lastsize-1 are the same
                    if last_list[i][j] == last_list[k][j] and last_list[i][lastsize - 1]  != last_list[k][lastsize - 1]:
                        # check the last element arent the same
                        # tmp.append(last_list[i][j])
                        tmp += (last_list[i][j],)
                        if j + 1 == lastsize - 1 and len(tmp) == lastsize - 1:
                            # print(len(tmp))
                            # tmp.append(last_list[i][lastsize - 1])
                            # tmp.append(last_list[k][lastsize - 1])
                            tmp += (last_list[i][lastsize - 1], last_list[k][lastsize - 1])
                    j += 1
            # end of j
            # print(tmp)
            if len(tmp) == currentsize:
                # print(tmp)
                current_list.append(tmp)
        # end of k   
        # if len(ll) > 0:    
        #     # print(ll) 
        #     t = []
        #     t.append(ll)
        #     current_list.append(t) 
    # end of i         
    # print(current_list)  
    return current_list
# end of function

def check_patients(current_list, currentsize):
    # list to return
    retlist = [[]]
    pos = 0
    # go through the list
    for i in range(len(current_list)):
        fullcount = 0
        print('\n')
        for j in range(len(current_list[0])): 
            counter = 0
            for k in range(hw11df.shape[0]):
                if hw11df.loc[k, current_list[i][j]] == 1:
                    counter += 1
            if counter >= 4:
                fullcounter += 1
        # if fullcount == currentsize:
            # print(current_list[i])
            retlist[0].append(current_list[i])
            print(retlist)
        # if current_list[i][0] == current_list[j][0] and current_list[i][1] != dfFN[0][j][1]:
        #     counter = 0
        #     for k in range(hw11df.shape[0]):
        #         x = current_list[i][0]
        #         y = current_list[i][1]
        #         z = current_list[j][1]
        #         if hw11df.loc[k, x] == 1 and hw11df.loc[k, y] == 1 and hw11df.loc[k, z] == 1:
        #             counter += 1
        #         if counter >= 4:
        #             trip = (current_list[i][0], current_list[i][1], current_list[j][1])
        #             retlist.append(trip)
    # if len(retlist) > 0:
    #     print(len(retlist))
    # print(retlist)
    return retlist
# end of function

In [301]:
currentsize = 3
lastsize = 2
tmp = dfFN
pos = 0

while(len(tmp[pos]) > currentsize):
    print(len(tmp[pos]))
    # get a list of N features
    last_list = tmp[pos]

    # get list of features for FN
    current_list = get_feature(last_list, lastsize)
    
    # go through and see if theyre in 4 patients
    if len(current_list) > 0:
        checked = check_patients(current_list, currentsize)
        break;
        # print(len(checked))
        # new_list = []
        # for i in range(len(checked)):
        #     for k in range(1, len(checked)):
        #         counter = 0
        #         for j in range(currentsize - 1):
        #             for z in range(currentsize - 1):
        #                 if checked[i][j] == checked[k][z]:
        #                     counter += 1
        #         if counter == 3:
        #                 new_list.append(checked[i])
        # print(len(list(set(checked) - set(new_list))))
        # print(new_list)
        # if len(new_list) >= currentsize + 1:
        #     tmp.append(new_list)
        #     # increment counters
        #     pos += 1
        #     currentsize += 1
        #     lastsize += 1
        # else:
        #     break;
    else:
        break;
# # end of while


122




UnboundLocalError: local variable 'fullcounter' referenced before assignment

In [427]:
dff5

,Association,Support,Confidence,SupportxConfidence
0,TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_...,0.053571,1.00,0.053571
1,TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_...,0.053571,0.75,0.040179
2,TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_...,0.053571,1.00,0.053571
3,TVP23C_GRCh37_17:15441469-15441469_Intron_SNP_...,0.053571,0.75,0.040179
4,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.053571,1.00,0.053571
5,ZBTB20_GRCh37_3:114058003-114058003_Frame-Shif...,0.053571,1.00,0.053571
6,DOCK3_GRCh37_3:51417604-51417604_Frame-Shift-D...,0.053571,0.75,0.040179
7,LARP4B_GRCh37_10:890939-890939_Frame-Shift-Del...,0.053571,1.00,0.053571
